In [ ]:
import random
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np
import math
import random
import unidecode
import datetime
import time
from time import strptime
import math
from itertools import chain
from IPython.core.display import display, HTML
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
display(HTML("<style>.container { width:90% !important; }</style>"))

# Below are dicts, lists, and strings that will be used in later functions
header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinnati',
                        'PIT': 'Pittsburgh',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SF': 'San Francisco',
                        'COL': 'Colorado',
                        'SD': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TB': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KC': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinnati',
                        'pirates': 'Pittsburgh',
                        'dodgers': 'Los Angeles Dodgers',
                        'diamondbacks': 'Arizona',
                        'd-backs': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}
city_to_nickname_dict ={'Atlanta': 'braves',
 'Washington': 'nationals',
 'New York Mets': 'mets',
 'Philadelphia': 'phillies',
 'Miami': 'marlins',
 'St. Louis': 'cardinals',
 'Milwaukee': 'brewers',
 'Chicago Cubs': 'cubs',
 'Cincinnati': 'reds',
 'Pittsburgh': 'pirates',
 'Los Angeles Dodgers': 'dodgers',
 'Arizona': 'd-backs',
 'San Francisco': 'giants',
 'Colorado': 'rockies',
 'San Diego': 'padres',
 'New York Yankees': 'yankees',
 'Tampa Bay': 'rays',
 'Boston': 'red sox',
 'Toronto': 'blue jays',
 'Baltimore': 'orioles',
 'Minnesota': 'twins',
 'Cleveland': 'indians',
 'Chicago White Sox': 'white sox',
 'Kansas City': 'royals',
 'Detroit': 'tigers',
 'Houston': 'astros',
 'Oakland': 'athletics',
 'Texas': 'rangers',
 'Los Angeles Angels': 'angels',
 'Seattle': 'mariners'}
league = {'Atlanta': 'NL',
          'Washington': 'NL',
          'New York Mets': 'NL',
          'Philadelphia': 'NL',
          'Miami': 'NL',
          'St. Louis': 'NL',
          'Milwaukee': 'NL',
          'Chicago Cubs': 'NL',
          'Cincinnati': 'NL',
          'Pittsburgh': 'NL',
          'Los Angeles Dodgers': 'NL',
          'Arizona': 'NL',
          'San Francisco': 'NL',
          'Colorado': 'NL',
          'San Diego': 'NL',
          'New York Yankees': 'AL',
          'Tampa Bay': 'AL',
          'Boston': 'AL',
          'Toronto': 'AL',
          'Baltimore': 'AL',
          'Minnesota': 'AL',
          'Cleveland': 'AL',
          'Chicago White Sox': 'AL',
          'Kansas City': 'AL',
          'Detroit': 'AL',
          'Houston': 'AL',
          'Oakland': 'AL',
          'Texas': 'AL',
          'Los Angeles Angels': 'AL',
          'Seattle': 'AL'}
team_num_dict = {'Atlanta': 16,
          'Washington': 24,
          'New York Mets': 25,
          'Philadelphia': 26,
          'Miami': 20,
          'St. Louis': 28,
          'Milwaukee': 23,
          'Chicago Cubs': 17,
          'Cincinnati': 18,
          'Pittsburgh': 27,
          'Los Angeles Dodgers': 22,
          'Arizona': 15,
          'San Francisco': 30,
          'Colorado': 19,
          'San Diego': 29,
          'New York Yankees': 9,
          'Tampa Bay': 12,
          'Boston': 3,
          'Toronto': 14,
          'Baltimore': 2,
          'Minnesota': 8,
          'Cleveland': 5,
          'Chicago White Sox': 4,
          'Kansas City': 7,
          'Detroit': 6,
          'Houston': 21,
          'Oakland': 10,
          'Texas': 13,
          'Los Angeles Angels': 1,
          'Seattle': 11}
mlb_teams = ['braves', 'nationals', 'mets', 'phillies', 'marlins', 
         'cardinals', 'brewers', 'cubs', 'reds', 'pirates', 
         'dodgers', 'd-backs', 'giants', 'rockies', 'padres', 
         'yankees', 'rays', 'red sox', 'blue jays', 'orioles', 'twins',
         'indians', 'white sox', 'royals', 'tigers', 'astros', 'athletics',
         'rangers', 'angels', 'mariners']

In [ ]:
class Batter_new:
    """Class that holds information for a Batter.
    
    Attributes:

    """
    def __init__(self, team, name, playerID, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L, singles_R, 
                 doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R,
                 singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                 doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, sb_rate, sb_success):
        """Initializes values for this class
        
        Args:
            team: team name of batter
            name: batter name
            playerID: fangraphs playerID
            hand: Batting hand of hitter(L, R, or S)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            hbp_L: fraction of plate appearances ending in a hbp vs. lefties
            so_L: fraction of plate appearances ending in a strikeout vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            hbp_R: fraction of plate appearances ending in a hbp vs. righties
            so_R: fraction of plate appearances ending in a strikeout vs. righties
            sb_rate: approximate percentage of time player will steal base given opportunity
            sb_success: fraction of stolen base attempts ending in success
            bsr: fangraphs stat representing projected runs created by a base runner. Higher number is better
            fld: bsr: fangraphs stat representing projected runs created by a fielder. higher number is better
        """
        self.team = team
        self.name = name
        self.playerID = playerID
        self.hand = hand
        self.singles_L = singles_L
        self.doubles_L = doubles_L
        self.triples_L = triples_L
        self.home_runs_L = home_runs_L
        self.walks_L = walks_L
        self.hbp_L = hbp_L
        self.so_L = so_L
        self.slg_L = walks_L+singles_L+2*doubles_L+3*triples_L+4*home_runs_L
        self.singles_R = singles_R
        self.doubles_R = doubles_R
        self.triples_R = triples_R
        self.home_runs_R = home_runs_R
        self.walks_R = walks_R
        self.hbp_R = hbp_R
        self.so_R = so_R
        self.slg_R = walks_R+singles_R+2*doubles_R+3*triples_R+4*home_runs_R
        self.singles_Home = singles_Home
        self.doubles_Home = doubles_Home
        self.triples_Home = triples_Home
        self.home_runs_Home = home_runs_Home
        self.walks_Home = walks_Home
        self.hbp_Home = hbp_Home
        self.so_Home = so_Home
        self.slg_Home = walks_Home+singles_Home+2*doubles_Home+3*triples_Home+4*home_runs_Home
        self.singles_Away = singles_Away
        self.doubles_Away = doubles_Away
        self.triples_Away = triples_Away
        self.home_runs_Away = home_runs_Away
        self.walks_Away = walks_Away
        self.hbp_Away = hbp_Away
        self.so_Away = so_Away
        self.slg_Away = walks_Away+singles_Away+2*doubles_Away+3*triples_Away+4*home_runs_Away
        self.sb_rate = sb_rate
        self.sb_success = sb_success 
        
        #self.cum values represent cumulative (season) totals for a batter
        self.cum_singles_L = 0
        self.cum_doubles_L = 0
        self.cum_triples_L = 0
        self.cum_home_runs_L = 0
        self.cum_walks_L = 0
        self.cum_hbp_L = 0
        self.cum_so_L = 0
        self.cum_pa_L = 0
        self.cum_singles_R = 0
        self.cum_doubles_R = 0
        self.cum_triples_R = 0
        self.cum_home_runs_R = 0
        self.cum_walks_R = 0
        self.cum_hbp_R = 0
        self.cum_so_R = 0
        self.cum_pa_R = 0
        self.cum_runs = 0
        self.cum_rbi = 0
        self.cum_sb = 0
        self.cum_cs = 0
        self.cum_sf = 0
        self.cum_sh = 0
        self.cum_gdp = 0
        self.cum_G = 0
        self.cum_GS = 0
     
    def hits(self):
        """cumulative hits of a batter
        
        Returns:
            int: number of cumulative hits
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L+
               self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf+self.cum_sh)>0):
            return(round(float((self.hits())/((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf))), 3))
        else:
            return 0    
        
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)>0):
            return(round((self.hits()+(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R))/(self.cum_pa_L+self.cum_pa_R), 3))
        else:
            return 0
        
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf)>0):
            return(round(((self.cum_singles_L+self.cum_singles_R)+2*(self.cum_doubles_L+self.cum_doubles_R)+
                           3*(self.cum_triples_L+self.cum_triples_R)+4*(self.cum_home_runs_L+self.cum_home_runs_R))/
                           ((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R+self.cum_hbp_L+self.cum_hbp_R+self.cum_sf+self.cum_sh)), 3))
        else:
            return 0
        
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form
            """
        return(self.slg() + self.obp())
    
    def strikeouts(self):
        """Number of strikeouts of a batter
        
        Returns:
            float: total strikeouts
        """
        return(self.cum_so_L+self.cum_so_R)
    
    def print_stats(self):
        """Prints some stats of a player
        """
        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str((self.cum_home_runs_L+self.cum_home_runs_R)))
              
    def hits_L(self):
        """cumulative hits of a batter against lefties
        
        Returns:
            int: number of hits of batter agianst lefties
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L)
    
    def avg_L(self):
        """Calculates avg of a player against lefties
        
        Returns:
            float: batting average in decimal form of batter against lefties
        """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(float((self.hits_L())/(self.cum_pa_L-self.cum_walks_L)), 3))
        else:
            return 0    
              
    def obp_L(self):
        """Calculates on base percentage of a player against lefties
        
        Returns:
            float: opb in decimal form of batter against lefties
        """
        if(self.cum_pa_L>0):
            return(round((self.hits_L()+self.cum_walks_L)/self.cum_pa_L, 3))
        else:
            return 0
              
    def slg_L(self):
        """Calculates slugging percentage of a player against lefties
        
        Returns:
            float: slg in decimal form of batter against lefties
            """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(((self.cum_singles_L+2*self.cum_doubles_L+
                           3*self.cum_triples_L+4*self.cum_home_runs_L))/
                           (self.cum_pa_L-self.cum_walks_L), 3))
        else:
            return 0
              
    def ops_L(self):
        """Calculates on base plus slugging of a player against lefties
        
        Returns:
            float: ops in decimal form of batter against lefties
        """
        return(self.slg_L() + self.obp_L())
              
    def hits_R(self):
        """Cumulative hits of a batter against righties
        
        Returns:
            int: number of hits of batter against righties
        """
        return(self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg_R(self):
        """Calculates avg of a player against righties
        
        Returns:
            float: batting average in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(float((self.hits_R())/(self.cum_pa_R-self.cum_walks_R)), 3))
        else:
            return 0    
              
    def obp_R(self):
        """Calculates on base percentage of a player against righties
        
        Returns:
            float: opb in decimal form of batter against righties
        """
        if(self.cum_pa_R>0):
            return(round((self.hits_R()+self.cum_walks_R)/self.cum_pa_R, 3))
        else:
            return 0
              
    def slg_R(self):
        """Calculates slugging percentage of a player against righties
        
        Returns:
            float: slg in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(((self.cum_singles_R2*self.cum_doubles_R+
                           3*self.cum_triples_R+4*self.cum_home_runs_R))/
                           (self.cum_pa_R-self.cum_walks_R), 3))
        else:
            return 0
              
    def ops_R(self):
        """Calculates on base plus slugging of a player against righties
        
        Returns:
            float: ops in decimal form of batter against righties
        """
        
        return(self.slg_R() + self.obp_R())
              
class Pitcher_new(Batter_new):
    """Class that holds information for a pitcher. Many attributes are similar to those from the Batter class.
    
    Attributes:

    """
    def __init__(self, team, name, playerID, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L, singles_R,
                 doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R,
                 singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                 doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, sb_rate, sb_success):
        """Initializes values for this class
        
        Args:
            team: team name of pitcher
            name: pitcher name
            hand: Pitching arm of pitcher(L, R)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            hbp_L: fraction of plate appearances ending in a hbp vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            hbp_R: fraction of plate appearances ending in a hbp vs. righties
            sb_rate: approximate percentage of time opposing player will steal base given opportunity (set to 0)
            sb_success: fraction of opponent stolen base attempts ending in success (set to 0)
        """
        
        Batter_new.__init__(self, team, name, playerID, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, hbp_L, so_L,
                        singles_R, doubles_R, triples_R, home_runs_R, walks_R, hbp_R, so_R,
                        singles_Home, doubles_Home, triples_Home, home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, 
                        doubles_Away, triples_Away, home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)
        self.cum_outs_L = 0
        self.cum_outs_R = 0
        self.cum_wp = 0
        self.current_bf = 0
        self.current_outs = 0
        self.current_runs = 0
        self.finished_half = False
        self.days_off = 100
        self.consecutive_days = 0
    
    def cum_outs(self, hand):
        """Cumulative outs of a pitcher against batters of certain handedness
        
        Args:
            hand: handedness of batter pitcher is facing
        Returns:
            float: value for cumulative outs
        """
        if(hand=='L'):
            return self.cum_outs_L
        elif(hand=='R'):
            return self.cum_outs_R
    
    def ip(self):
        """Calculates innings pitched of a pitcher
        
        Returns:
            float: Innings pitched in normal form(ie. 5.1 for 5 and a third innings pitched)
        """
        ip = (self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3.0
        ip = math.modf(ip)
        ip = round((ip[0]/3.33333 + ip[1]), 2)
        return(ip)  
    
    def era(self):
        """Calculates ERA of a pitcher
        
        Returns:
            float: ERA in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((27*self.cum_runs/(self.cum_outs_L+self.cum_outs_R+self.cum_cs)), 2))
    
    def whip(self):
        """Calculates WHIP of a pitcher
        
        Returns:
            float: WHIP in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((((self.cum_walks_L+self.cum_walks_R)+self.hits())/
                          ((self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3)), 2))

class Team_new:
    """Class that holds information for a team.
    
    Attributes:

    """
    def __init__(self, city, league, batter_df, pitcher_df, roster, batter_stats, pitcher_stats, lineup_dict=None, lineup_id_dict=None):
        """Initializes values for this class
        
        Args:
            city: city of team (ex. 'Atlanta')
            league: league of team ('AL' or 'NL')
            batter_df: dataframe of batters on a team and their 2019 stats
            pitcher_df: dataframe of pitchers on a team and their 2019 stats
            roster: team's roster
            batter_stats: stats for all batter in MLB
            pitcher_stats: stats for all pitchers in MLB
            lineup_dict: team's lineup of player names
            lineup_id_dict: team's lineup of playerID's
        """
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.batter_dict= {}
        self.pitcher_df = pitcher_df
        self.pitcher_dict = {}
        self.roster = roster
        self.batter_stats = batter_stats
        self.pitcher_stats = pitcher_stats
        self.lineup_dict = lineup_dict
        self.lineup_id_dict = lineup_id_dict
        self.opener = None
        self.probable = None   # Probable pitcher if team has opener
        self.pitchers = []   # List of pithcers on a team
        self.rotation = []   # Team's starting rotation
        self.bullpen = []    # Team's bullpen  
        self.starter = None   # Starter for a team
        self.closer = None    # Clsoer for a team
        self.batters = []    # List of batters on a team
        self.lineup = [None]*9    # A team's lineup
        self.game_lineup = []     # A team's lineup that may change in simulated game
        self.bench = []      # Bench for a team
        self.current_pitcher_list = []     # pitchers who have already pitched in a given game
        self.available_pitchers_list = []   # Pitchers who have not pitched yet in a given game
        self.batting_index = 0   # Where team is at in lineup
        self.runners = [None, None, None, 0]    # first base-home plate. 0 for base empty, 1 for runner on base
        self.runs = 0   # Number of runs for team in a given game
        self.runners_dict = {}
        self.game_no = 0    # game number
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        
        avg_singles_L = self.batter_stats['singles_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_doubles_L = self.batter_stats['doubles_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_triples_L = self.batter_stats['triples_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_home_runs_L = self.batter_stats['HR_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_walks_L = self.batter_stats['BB_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_hbp_L = self.batter_stats['HBP_L'].mean()/self.batter_stats['PA_L'].mean()
        avg_so_L = self.batter_stats['SO_L'].mean()/self.batter_stats['PA_L'].mean()
        
        avg_singles_R = self.batter_stats['singles_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_doubles_R = self.batter_stats['doubles_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_triples_R = self.batter_stats['triples_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_home_runs_R = self.batter_stats['HR_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_walks_R = self.batter_stats['BB_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_hbp_R = self.batter_stats['HBP_R'].mean()/self.batter_stats['PA_R'].mean()
        avg_so_R = self.batter_stats['SO_R'].mean()/self.batter_stats['PA_R'].mean()
        
        avg_singles_Home = self.batter_stats['singles_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_doubles_Home = self.batter_stats['doubles_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_triples_Home = self.batter_stats['triples_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_home_runs_Home = self.batter_stats['HR_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_walks_Home = self.batter_stats['BB_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_hbp_Home = self.batter_stats['HBP_Home'].mean()/self.batter_stats['PA_Home'].mean()
        avg_so_Home = self.batter_stats['SO_Home'].mean()/self.batter_stats['PA_Home'].mean()
        
        avg_singles_Away = self.batter_stats['singles_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_doubles_Away = self.batter_stats['doubles_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_triples_Away = self.batter_stats['triples_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_home_runs_Away = self.batter_stats['HR_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_walks_Away = self.batter_stats['BB_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_hbp_Away = self.batter_stats['HBP_Away'].mean()/self.batter_stats['PA_Away'].mean()
        avg_so_Away = self.batter_stats['SO_Away'].mean()/self.batter_stats['PA_Away'].mean()

        for index, batter in self.batter_df.iterrows():   # adding values to each batter and instantiating Batter class
            singles_L = 2*batter['singles_L']/batter['PA_L']-avg_singles_L
            doubles_L = 2*batter['doubles_L']/batter['PA_L']-avg_doubles_L
            triples_L = 2*batter['triples_L']/batter['PA_L']-avg_triples_L
            home_runs_L = 2*batter['HR_L']/batter['PA_L']-avg_home_runs_L
            walks_L = 2*batter['BB_L']/batter['PA_L']-avg_walks_L
            hbp_L = 2*batter['HBP_L']/batter['PA_L']-avg_hbp_L
            so_L = 2*batter['SO_L']/batter['PA_L']-avg_so_L
            
            singles_R = 2*batter['singles_R']/batter['PA_R']-avg_singles_R
            doubles_R = 2*batter['doubles_R']/batter['PA_R']-avg_doubles_R
            triples_R = 2*batter['triples_R']/batter['PA_R']-avg_triples_R
            home_runs_R = 2*batter['HR_R']/batter['PA_R']-avg_home_runs_R
            walks_R = 2*batter['BB_R']/batter['PA_R']-avg_walks_R
            hbp_R = 2*batter['HBP_R']/batter['PA_R']-avg_hbp_R
            so_R = 2*batter['SO_R']/batter['PA_R']-avg_so_R
            
            singles_Home = 2*batter['singles_Home']/batter['PA_Home']-avg_singles_Home
            doubles_Home = 2*batter['doubles_Home']/batter['PA_Home']-avg_doubles_Home
            triples_Home = 2*batter['triples_Home']/batter['PA_Home']-avg_triples_Home
            home_runs_Home = 2*batter['HR_Home']/batter['PA_Home']-avg_home_runs_Home
            walks_Home = 2*batter['BB_Home']/batter['PA_Home']-avg_walks_Home
            hbp_Home = 2*batter['HBP_Home']/batter['PA_Home']-avg_hbp_Home
            so_Home = 2*batter['SO_Home']/batter['PA_Home']-avg_so_Home
            
            singles_Away = 2*batter['singles_Away']/batter['PA_Away']-avg_singles_Away
            doubles_Away = 2*batter['doubles_Away']/batter['PA_Away']-avg_doubles_Away
            triples_Away = 2*batter['triples_Away']/batter['PA_Away']-avg_triples_Away
            home_runs_Away = 2*batter['HR_Away']/batter['PA_Away']-avg_home_runs_Away
            walks_Away = 2*batter['BB_Away']/batter['PA_Away']-avg_walks_Away
            hbp_Away = 2*batter['HBP_Away']/batter['PA_Away']-avg_hbp_Away
            so_Away = 2*batter['SO_Away']/batter['PA_Away']-avg_so_Away
            
            current_batter = Batter_new(self.city, batter['Name'], batter['playerID'], batter['Bats'], singles_L, doubles_L, triples_L, 
                                home_runs_L, walks_L, hbp_L, so_L, singles_R, doubles_R, triples_R, 
                                home_runs_R, walks_R, hbp_R, so_R, singles_Home, doubles_Home, triples_Home, 
                                home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, doubles_Away, triples_Away, 
                                home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)

            if batter['playerID'] in self.lineup_id_dict['lineup']:   # Adding batters to lineup
                self.lineup[self.lineup_id_dict['lineup'].index(batter['playerID'])] = current_batter
            else:  # Adding batters not in lineup to bench
                self.bench.append(current_batter)
            
            self.batters.append(current_batter)
            self.batter_dict.update({batter['Name']: current_batter})
            
        for num in range(len(self.lineup)):  # If normal batter in lineup is not in dataset, add fake batter
            if self.lineup[num]==None:
                
                if(num==8 and self.league=='NL'):   # If pitcher's spot, add poor-hitting pitcher
                    self.lineup[8] = Batter_new(self.city, 'pitcher', -1, 'R', .06, .01, -.05, -.03, -.03, 0, .5, .06, .01, 
                                                  -.05, -.03, -.03, 0, .5, .06, .01, -.05, -.03, -.03, 0, .5, .06, .01, 
                                                  -.05, -.03, -.03, 0, .5, 0, 0)
                else:    # Add replacement level batter
                    self.lineup[num] = Batter_new(self.city, 'extra' + str(num), -1, 'R', .14, .04, .005, .02, 
                                              .04, .01, .14, .04, .005, .02, .04, .01, .02, .01, .14, .04, .005, .02, 
                                              .04, .01, .14, .04, .005, .02, .04, .01, .02, .01, 0, 0)
                    
                self.batters.append(self.lineup[num])
                print("batter_name", self.lineup[num].name)
            else:
                print("batter_name", self.lineup[num].name)
                
        self.game_lineup = self.lineup.copy()
        
        avg_singles_L = self.pitcher_stats['singles_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_doubles_L = self.pitcher_stats['doubles_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_triples_L = self.pitcher_stats['triples_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_home_runs_L = self.pitcher_stats['HR_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_walks_L = self.pitcher_stats['BB_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_hbp_L = self.pitcher_stats['HBP_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        avg_so_L = self.pitcher_stats['SO_L'].mean()/self.pitcher_stats['TBF_L'].mean()
        
        avg_singles_R = self.pitcher_stats['singles_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_doubles_R = self.pitcher_stats['doubles_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_triples_R = self.pitcher_stats['triples_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_home_runs_R = self.pitcher_stats['HR_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_walks_R = self.pitcher_stats['BB_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_hbp_R = self.pitcher_stats['HBP_R'].mean()/self.pitcher_stats['TBF_R'].mean()
        avg_so_R = self.pitcher_stats['SO_R'].mean()/self.pitcher_stats['TBF_R'].mean()  
        
        avg_singles_Home = self.pitcher_stats['singles_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_doubles_Home = self.pitcher_stats['doubles_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_triples_Home = self.pitcher_stats['triples_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_home_runs_Home = self.pitcher_stats['HR_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_walks_Home = self.pitcher_stats['BB_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_hbp_Home = self.pitcher_stats['HBP_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        avg_so_Home = self.pitcher_stats['SO_Home'].mean()/self.pitcher_stats['TBF_Home'].mean()
        
        avg_singles_Away = self.pitcher_stats['singles_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_doubles_Away = self.pitcher_stats['doubles_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_triples_Away = self.pitcher_stats['triples_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_home_runs_Away = self.pitcher_stats['HR_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_walks_Away = self.pitcher_stats['BB_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_hbp_Away = self.pitcher_stats['HBP_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        avg_so_Away = self.pitcher_stats['SO_Away'].mean()/self.pitcher_stats['TBF_Away'].mean()
        
            
        for index, pitcher in pitcher_df.iterrows(): # adding values to each pitcher and instantiating Pitcher class
            singles_L = 2*pitcher['singles_L']/pitcher['TBF_L']-avg_singles_L
            doubles_L = 2*pitcher['doubles_L']/pitcher['TBF_L']-avg_doubles_L
            triples_L = 2*pitcher['triples_L']/pitcher['TBF_L']-avg_triples_L
            home_runs_L = 2*pitcher['HR_L']/pitcher['TBF_L']-avg_home_runs_L
            walks_L = 2*pitcher['BB_L']/pitcher['TBF_L']-avg_walks_L
            hbp_L = 2*pitcher['HBP_L']/pitcher['TBF_L']-avg_hbp_L
            so_L = 2*pitcher['SO_L']/pitcher['TBF_L']-avg_so_L
            
            singles_R = 2*pitcher['singles_R']/pitcher['TBF_R']-avg_singles_R
            doubles_R = 2*pitcher['doubles_R']/pitcher['TBF_R']-avg_doubles_R
            triples_R = 2*pitcher['triples_R']/pitcher['TBF_R']-avg_triples_R
            home_runs_R = 2*pitcher['HR_R']/pitcher['TBF_R']-avg_home_runs_R
            walks_R = 2*pitcher['BB_R']/pitcher['TBF_R']-avg_walks_R
            hbp_R = 2*pitcher['HBP_R']/pitcher['TBF_R']-avg_hbp_R
            so_R = 2*pitcher['SO_R']/pitcher['TBF_R']-avg_so_R
            
            singles_Home = 2*pitcher['singles_Home']/pitcher['TBF_Home']-avg_singles_Home
            doubles_Home = 2*pitcher['doubles_Home']/pitcher['TBF_Home']-avg_doubles_Home
            triples_Home = 2*pitcher['triples_Home']/pitcher['TBF_Home']-avg_triples_Home
            home_runs_Home = 2*pitcher['HR_Home']/pitcher['TBF_Home']-avg_home_runs_Home
            walks_Home = 2*pitcher['BB_Home']/pitcher['TBF_Home']-avg_walks_Home
            hbp_Home = 2*pitcher['HBP_Home']/pitcher['TBF_Home']-avg_hbp_Home
            so_Home = 2*pitcher['SO_Home']/pitcher['TBF_Home']-avg_so_Home
            
            singles_Away = 2*pitcher['singles_Away']/pitcher['TBF_Away']-avg_singles_Away
            doubles_Away = 2*pitcher['doubles_Away']/pitcher['TBF_Away']-avg_doubles_Away
            triples_Away = 2*pitcher['triples_Away']/pitcher['TBF_Away']-avg_triples_Away
            home_runs_Away = 2*pitcher['HR_Away']/pitcher['TBF_Away']-avg_home_runs_Away
            walks_Away = 2*pitcher['BB_Away']/pitcher['TBF_Away']-avg_walks_Away
            hbp_Away = 2*pitcher['HBP_Away']/pitcher['TBF_Away']-avg_hbp_Away
            so_Away = 2*pitcher['SO_Away']/pitcher['TBF_Away']-avg_so_Away

            current_pitcher = Pitcher_new(self.city, pitcher['Name'], pitcher['playerID'], pitcher['Throws'], singles_L, doubles_L, triples_L, 
                                  home_runs_L, walks_L, hbp_L, so_L, singles_R, doubles_R, triples_R, 
                                  home_runs_R, walks_R, hbp_R, so_R, singles_Home, doubles_Home, triples_Home, 
                                  home_runs_Home, walks_Home, hbp_Home, so_Home, singles_Away, doubles_Away, triples_Away, 
                                  home_runs_Away, walks_Away, hbp_Away, so_Away, 0, 0)

            if 'probable' in self.lineup_dict.keys() and self.lineup_id_dict['probable']!=None and (pitcher['playerID']==self.lineup_id_dict['probable']):  # Assigning starter to the team
                self.probable = current_pitcher
            elif 'opener' in self.lineup_dict.keys() and self.lineup_id_dict['opener']!=None and (pitcher['playerID']==self.lineup_id_dict['opener']):  # Assigning starter to the team
                self.opener = current_pitcher
            
            if 'starter' in self.lineup_dict.keys() and self.lineup_id_dict['starter']!=None and (pitcher['playerID']==self.lineup_id_dict['starter']):  # Assigning starter to the team
                self.starter = current_pitcher

            if 'SP' in self.roster.keys() and pitcher['playerID'] in self.roster['SP']:
                self.rotation.append(current_pitcher)
                
            if 'RP' in self.roster.keys() and pitcher['playerID'] in self.roster['RP'] and pitcher['playerID'] not in self.roster['SP']:
                self.bullpen.append(current_pitcher)
                
            if 'Closer' in self.roster.keys() and pitcher['playerID']==self.roster['Closer']:
                self.closer = current_pitcher
            
            self.pitchers.append(current_pitcher)
            self.pitcher_dict.update({pitcher['Name']: current_pitcher})
            
        if self.starter==None:   # Fake starter if not in dataset
            self.starter = Pitcher_new(self.city, 'fake_starter', -1, 'R', avg_singles_L*1.1,  avg_doubles_L*1.1, avg_triples_L*1.1, 
                              avg_home_runs_L*1.1, avg_walks_L*1.1, avg_hbp_L*1.1, avg_so_L*0.9, avg_singles_R*1.1, avg_doubles_R*1.1, avg_triples_R*1.1, 
                              avg_home_runs_R*1.1, avg_walks_R*1.1, avg_hbp_R*1.1, avg_so_R*0.9, avg_singles_Home*1.1, avg_doubles_Home*1.1, avg_triples_Home*1.1, 
                              avg_home_runs_Home*1.1, avg_walks_Home*1.1, avg_hbp_Home*1.1, avg_so_Home*0.9, avg_singles_Away*1.1, avg_doubles_Away*1.1, avg_triples_Away*1.1, 
                              avg_home_runs_Away*1.1, avg_walks_Away*1.1, avg_hbp_Away*1.1, avg_so_Away*0.9, 0, 0)
            self.pitchers.append(self.starter)
            
        if self.lineup_dict['probable']!=None and self.probable==None:   # Fake probable if in game but not in dataset
            self.probable = Pitcher_new(self.city, 'fake_probable', -1, 'R', avg_singles_L*1.1,  avg_doubles_L*1.1, avg_triples_L*1.1, 
                              avg_home_runs_L*1.1, avg_walks_L*1.1, avg_hbp_L*1.1, avg_so_L*0.9, avg_singles_R*1.1, avg_doubles_R*1.1, avg_triples_R*1.1, 
                              avg_home_runs_R*1.1, avg_walks_R*1.1, avg_hbp_R*1.1, avg_so_R*0.9, avg_singles_Home*1.1, avg_doubles_Home*1.1, avg_triples_Home*1.1, 
                              avg_home_runs_Home*1.1, avg_walks_Home*1.1, avg_hbp_Home*1.1, avg_so_Home*0.9, avg_singles_Away*1.1, avg_doubles_Away*1.1, avg_triples_Away*1.1, 
                              avg_home_runs_Away*1.1, avg_walks_Away*1.1, avg_hbp_Away*1.1, avg_so_Away*0.9, 0, 0)
            self.pitchers.append(self.probable)
                   
    def choose_pitcher(self, outs, inning, innings, runs, opp_runs, hitting_lineup, hitting_lineup_index, pitchers_lineup, pitchers_lineup_index):
        """Returns a pitcher to pitch for a team given game attributes
        
            Args:
                outs: current number of outs in inning
                runs: number of runs team has
                opp_runs: number of runs opposing team has
                hitting_lineup: batting team's lineup
                hitting_lineup_index: place in lineup of batting team
                pitchers_lineup: fielding team's opposing team's lineup
                pitchers_lineup_index: place in lineup of fielding team
            Returns:
                instance of Pitcher class to pitch in game
            """
        
        remove_pitcher = False
        self.available_pitchers_list = []
        for pitcher in self.pitchers:  # Adding pitchers who have not already pitched to a list
            if pitcher not in self.current_pitcher_list and pitcher not in self.available_pitchers_list:
                self.available_pitchers_list.append(pitcher)

        # If Pitcher hasn't faced the minimum of 3 batters or finished a hlaf inning, they remain in game
        if(self.current_pitcher.current_bf<3 and self.current_pitcher.finished_half==False): 
            return self.current_pitcher
        
        # Creating variable total that uses game factors to determine if pitcher should get taken out
        total = ((self.current_pitcher.current_bf-self.current_pitcher.current_outs)*3 + self.current_pitcher.current_outs+self.current_pitcher.current_runs*3)

        if(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3.0>0):
            total+=(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3)*2
        
        if(self.current_pitcher.hand==hitting_lineup[hitting_lineup_index].hand):   # If batter and pitcher have same handedness
            total-=3
            
        if(self.current_pitcher.finished_half):  # If pitcher has already finished half inning
            total+=2
        
        if(outs==0 and (self.current_pitcher in self.rotation or self.current_pitcher==self.starter)):    # If no outs yet in inning
            total+=4
            
        if(pitchers_lineup_index>4 and pitchers_lineup[8].name=='pitcher'):  # If pitcher is in lineup, spot is almost up
            total+=6
            
        if(self.current_pitcher==self.probable):   # When pitcher comes in after opener
            total+=6
        
        if(self.current_pitcher==self.opener and self.current_pitcher.finished_half):   # Opener should usually go only 1 inning
            total+=4
            
        if(innings==7 and self.current_pitcher==self.probable):   # Add more for pitchers in 7 inning games
            total+=6
        
        if(total<50):
            num = random.uniform(0, (60-total)*200)
            #print("num", str(num))
        else:
            num = random.uniform(0, (90-total)*4)
            
        
        if(num<total):
            remove_pitcher = True
        
        # Leave current pitcher in probabilistically
        if(not remove_pitcher and (((self.current_pitcher in self.rotation and self.current_pitcher==self.starter) 
           or self.current_pitcher==self.probable) and self.current_pitcher!=self.opener)):   
            return self.current_pitcher
        elif self.current_pitcher==self.opener and total<=13:   # Openers
            return self.current_pitcher       
        elif self.current_pitcher!=self.opener and self.current_pitcher!=self.probable and random.randint(round(total)-2, round(total)+2)<=8:  # Relievers
            return self.current_pitcher
        else:  # If current pitcher has total value greater than threshold, find new reliever
            #print("Removing pitcher: " + self.current_pitcher.name + ' with total: ' + str(total))
            if((inning==innings-1 and outs>0 and self.closer!=None and self.closer not in self.current_pitcher_list 
               and runs-opp_runs>0 and runs-opp_runs<3) or (inning>innings-1 and self.closer!=None 
               and self.closer not in self.current_pitcher_list and runs-opp_runs>0 and runs-opp_runs<4)): # Put closer in game if game is close and in late innings
                #print("closer coming in")
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            else:
                # If game not close or game not in late innings, put non-closer reliever in game
                
                left_count = 0    # Number of lefties out of next three batters
                right_count = 0   # Number of righties out of next three batters

                for num in range(3):
                    if(hitting_lineup[(hitting_lineup_index+num)%9].hand=='L'):
                        left_count+=(1+outs-num)
                    elif(hitting_lineup[(hitting_lineup_index+num)%9].hand=='R'):
                        right_count+=(1+outs-num)

                if(left_count>right_count):  # If more of the next 3 batters are lefties, hand_indicator = 0
                    hand_indicator = 'L'
                else:                        # If more or equal of the next 3 batters are righties, hand_indicator = 1
                    hand_indicator = 'R'

                relievers_left = []
                #print("bullpen")
                for pitcher in self.bullpen:
                    if pitcher in self.available_pitchers_list and self.closer!=pitcher:
                        relievers_left.append(pitcher)
                        #print(pitcher.name + ' ' + pitcher.hand)
                      
                if len(relievers_left)==0:
                    if self.closer in self.available_pitchers_list:
                        self.current_pitcher = self.closer
                        self.current_pitcher.cum_G+=1
                        self.current_pitcher_list.append(self.current_pitcher)
                        #print(self.current_pitcher.name + ' coming in closer no other relievers left')
                        return self.current_pitcher
                    else:
                        pitcher_chosen = False
                        attempts = 0
                        while(not pitcher_chosen and len(self.available_pitchers_list)>0 and attempts<200):
                            num = random.randint(0, len(self.available_pitchers_list)-1)
                            pitcher = self.available_pitchers_list[num]
                            if(pitcher in self.rotation and attempts<30 and pitcher.days_off<3):
                                attempts+=1
                            elif(pitcher in self.rotation and attempts<60 and pitcher.days_off<2):
                                attempts+=1
                            elif(pitcher in self.rotation and attempts<100 and pitcher.days_off<1):
                                attempts+=1
                            else:
                                self.current_pitcher = pitcher
                                self.current_pitcher.cum_G+=1
                                self.current_pitcher_list.append(self.current_pitcher)
                                #print(self.current_pitcher.name + ' coming in starter in no relievers left')
                                return self.current_pitcher
   
                elif len(relievers_left)==1:
                    self.current_pitcher = relievers_left[0]
                    self.current_pitcher.cum_G+=1
                    self.current_pitcher_list.append(self.current_pitcher)
                    #print(self.current_pitcher.name + ' coming in only non-closer left')
                    return self.current_pitcher
            
                else:
                    wrong_hand = 0
                    attempts = 0
                    
                    while(attempts<200):    # Trying to find a reliever

                        num = random.randint(0, len(relievers_left)-1)
                        pitcher = relievers_left[num]
                        total = 0
                        if(pitcher.hand==hand_indicator or wrong_hand>25):
                            self.current_pitcher = pitcher
                            self.current_pitcher.cum_G+=1
                            self.current_pitcher_list.append(self.current_pitcher)
                            #print(self.current_pitcher.name + ' coming in normal')
                            return self.current_pitcher

                        wrong_hand+=1
                        attempts+=1

        #print(self.current_pitcher.name + ' coming in end')
        return self.current_pitcher
           
    def restart(self, season=False):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started
        """
        self.game_lineup = self.lineup.copy()
        
        for pitcher in self.current_pitcher_list:
            pitcher.consecutive_days+=1
            pitcher.days_off = 0
            pitcher.current_bf = 0
            pitcher.current_outs = 0
            pitcher.current_runs = 0
            pitcher.finished_half = False
        
        for pitcher in self.available_pitchers_list:
            pitcher.days_off+=1
            pitcher.consecutive_days = 0

        self.runners_dict = {}    
        self.current_pitcher_list = []   
        self.batting_index = 0
        self.current_pitcher = None
        self.starter_pitching = True
        self.runners = [None, None, None, 0]
        self.runs = 0
        
        if season:
            self.wins = 0
            self.losses = 0
            self.cum_runs = 0
            self.cum_runs_allowed = 0
            self.game_no = 0
            
            for batter in self.batters:
                batter.cum_singles_L = 0
                batter.cum_doubles_L = 0
                batter.cum_triples_L = 0
                batter.cum_home_runs_L = 0
                batter.cum_walks_L = 0
                batter.cum_hbp_L = 0
                batter.cum_so_L = 0
                batter.cum_pa_L = 0
                batter.cum_singles_R = 0
                batter.cum_doubles_R = 0
                batter.cum_triples_R = 0
                batter.cum_home_runs_R = 0
                batter.cum_walks_R = 0
                batter.cum_hbp_R = 0
                batter.cum_so_R = 0
                batter.cum_pa_R = 0
                batter.cum_runs = 0
                batter.cum_rbi = 0
                batter.cum_sb = 0
                batter.cum_cs = 0
                batter.cum_sf = 0
                batter.cum_sh = 0
                batter.cum_gdp = 0
                batter.cum_G = 0
                batter.cum_GS = 0
                
            for pitcher in self.pitchers:
                pitcher.cum_singles_L = 0
                pitcher.cum_doubles_L = 0
                pitcher.cum_triples_L = 0
                pitcher.cum_home_runs_L = 0
                pitcher.cum_walks_L = 0
                pitcher.cum_hbp_L = 0
                pitcher.cum_so_L = 0
                pitcher.cum_pa_L = 0
                pitcher.cum_singles_R = 0
                pitcher.cum_doubles_R = 0
                pitcher.cum_triples_R = 0
                pitcher.cum_home_runs_R = 0
                pitcher.cum_walks_R = 0
                pitcher.cum_hbp_R = 0
                pitcher.cum_so_R = 0
                pitcher.cum_pa_R = 0
                pitcher.cum_runs = 0
                pitcher.cum_rbi = 0
                pitcher.cum_sb = 0
                pitcher.cum_cs = 0
                pitcher.cum_sf = 0
                pitcher.cum_sh = 0
                pitcher.cum_gdp = 0
                pitcher.cum_G = 0
                pitcher.cum_GS = 0
                pitcher.cum_outs_L = 0
                pitcher.cum_outs_R = 0
                pitcher.cum_wp = 0

    def reset_runners(self):
        """Resets baserunners after each half inning
        """
        self.runners = [None, None, None, 0]
    
    def cumul_stats_df(self):
        """Class that returns dataframes of stats for batters and pitchers
    
        Returns:
            batter_df: dataframe of batter stats
            pitcher_df: dataframe of pitcher stats
        """
        batter_df = pd.DataFrame(columns=['Name', 'G', 'PA', 'AB', 'H', 'singles', 'doubles', 'triples', 'HR', 'BB', 'HBP', 'SF', 
                                          'SH', 'K', 'AVG', 'OBP', 'SLG', 'OPS', 'R', 'RBI', 'SB', 'CS'])
        
        for b in self.batters:
            batter_values = [b.name, b.cum_G, b.cum_pa_L+b.cum_pa_R, b.cum_pa_L+b.cum_pa_R-
                             (b.cum_walks_L+b.cum_walks_R+b.cum_hbp_L+b.cum_hbp_R+b.cum_sf), 
                             b.hits(), b.cum_singles_L+b.cum_singles_R, b.cum_doubles_L+b.cum_doubles_R, 
                             b.cum_triples_L+b.cum_triples_R, b.cum_home_runs_L+b.cum_home_runs_R, 
                             b.cum_walks_L+b.cum_walks_R, b.cum_hbp_L+b.cum_hbp_R, b.cum_sf, b.cum_sh, b.strikeouts(), b.avg(), b.obp(), 
                             b.slg(), b.ops(), b.cum_runs, b.cum_rbi, b.cum_sb, b.cum_cs]
            batter_df = batter_df.append(pd.Series(batter_values, index=batter_df.columns), ignore_index=True)
        batter_df.sort_values(by=['PA'], ascending=False, inplace=True)
        
        pitcher_df = pd.DataFrame(columns=['Name', 'G', 'GS', 'IP', 'K', 'H', 'BB', 'HBP', 'ERA', 'WHIP', 'HR',
                                           'AVG', 'OBP', 'SLG', 'OPS', 'R', 'SB', 'CS', 'WP'])
        
        for p in self.pitchers:
            pitcher_values = [p.name, p.cum_G, p.cum_GS, p.ip(), p.strikeouts(), p.hits(), p.cum_walks_L+p.cum_walks_R, 
                              p.cum_hbp_L+p.cum_hbp_R, p.era(), p.whip(), p.cum_home_runs_L+p.cum_home_runs_R, p.avg(), 
                              p.obp(), p.slg(), p.ops(), p.cum_runs, p.cum_sb, p.cum_cs, p.cum_wp]
            pitcher_df = pitcher_df.append(pd.Series(pitcher_values, index=pitcher_df.columns), ignore_index=True)
        pitcher_df.sort_values(by=['IP'], ascending=False, inplace=True) 
        
        return batter_df, pitcher_df
    
class Game_new:
    """Class that holds information for a Game.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team, innings):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team
        self.innings = innings
                                
    def get_sb_wp_result(self, runners, pitcher, team, outs, num):
        """stolen base result of a batter
        
        Args:
            runners: list of runners on base
            pitcher: pitcher currently pitching
            team: batting team
            outs: number of outs
            num: random number used to determine whether or not batter should steal base
        Returns:
            result of stolen base attempt (or None if no attempt)
        """
        if(runners[0]!=None and runners[1]==None):   # If first base occupied and second base not, steal possible
            if(num>runners[0].sb_rate/1.2):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[0].sb_success):  # Successful stolen base attempt
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[0].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[0] = None
                    return "out"
        elif(runners[0]!=None and runners[1]!=None):   # If first and second base occupied
            if(num>runners[0].sb_rate/5):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[1].sb_success):  # Successful stolen base attempt
                    runners[1].cum_sb+=1
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=2
                    runners[2] = runners[1]
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[1].cum_cs+=1
                    pitcher.cum_cs+=1
                    if(outs<2):
                        runners[0].cum_sb+=1
                        pitcher.cum_sb+=1
                        runners[1] = runners[0]
                        
                    runners[0] = None
                    return "out"
        elif(runners[0]!=None and runners[1]!=None):   # If second base occupied
            if(num>runners[0].sb_rate/5):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[1].sb_success):  # Successful stolen base attempt
                    runners[1].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[2] = runners[1]
                    runners[1] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[1].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[1] = None
                    return "out"
        
        wp_num = random.uniform(0,1)
        if(wp_num<.01):    # Seeing whether at bat featured wild pitch
            pitcher.cum_wp+=1
            if(runners[2]!=None):
                team.runs+=1
                runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = None
        if(wp_num<.0005):    # Seeing whether at bat featured second wild pitch   
            pitcher.cum_wp+=1
            if(runners[2]!=None):
                team.runs+=1
                runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                
            team.runners[2] = team.runners[1]
            team.runners[1] = team.runners[0]
            team.runners[0] = None          
        
    def get_result(self, batter, pitcher, outs, runners, batting_side, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            outs: number of outs in inning
            runners: List of where runners are
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """      
        hand_multiplier = 3/4
        field_multiplier = 1-hand_multiplier
        
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        pitcher.current_bf+=1
        
        sacrifice_num = random.uniform(0,1)
        if(outs<2 and batter.name=='pitcher' and runners[2]==None and (runners[1]!=None or runners[2]==None)):    # Seeing whether a pitcher will try to lay down a sacrifice bunt
            if(sacrifice_num<.3):
                
                batter.cum_sh+=1
                
                if(batter_against=='L'):
                    batter.cum_pa_L+=1
                else:
                    batter.cum_pa_R+=1
                    
                if(pitcher_against=='L'):
                    pitcher.cum_pa_L+=1
                    pitcher.cum_outs_L+=1
                else:
                    pitcher.cum_pa_R+=1
                    pitcher.cum_outs_R+=1
                
                return 'sh'
            elif(sacrifice_num<.45):
                
                if(batter_against=='L'):
                    batter.cum_pa_L+=1
                else:
                    batter.cum_pa_R+=1
                    
                if(pitcher_against=='L'):
                    pitcher.cum_pa_L+=1
                    pitcher.cum_outs_L+=1
                else:
                    pitcher.cum_pa_R+=1
                    pitcher.cum_outs_R+=1
                
                return 'out'
            
        if(batter_against=='L' and pitcher_against=='L' and batting_side=='Home'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_L+pitcher.singles_L)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_multiplier):
                batter.cum_singles_L+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away)*field_multiplier):
                batter.cum_triples_L+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L
                      +batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L)*hand_multiplier
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                    +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away)*field_multiplier):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                     +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away)*field_multiplier):
                batter.cum_walks_L+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home
                     +pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home
                     +pitcher.walks_Away+batter.hbp_Home+pitcher.hbp_Away)*field_multiplier):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L+batter.so_L+pitcher.so_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away
                     +batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away
                     +batter.hbp_Home+pitcher.hbp_Away+batter.so_Home+pitcher.so_Away)*field_multiplier):
                batter.cum_so_L+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
                
        elif(batter_against=='L' and pitcher_against=='L' and batting_side=='Away'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_L+pitcher.singles_L)*hand_multiplier + (batter.singles_Away+pitcher.singles_Home)*field_multiplier):
                batter.cum_singles_L+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home)*field_multiplier):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home)*field_multiplier):
                batter.cum_triples_L+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L
                      +batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L)*hand_multiplier
                    + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                    +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home)*field_multiplier):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                     +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home)*field_multiplier):
                batter.cum_walks_L+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away
                     +pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away
                     +pitcher.walks_Home+batter.hbp_Away+pitcher.hbp_Home)*field_multiplier):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<(batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L
                     +batter.home_runs_L+pitcher.home_runs_L+batter.walks_L+pitcher.walks_L+batter.hbp_L+pitcher.hbp_L+batter.so_L+pitcher.so_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home
                     +batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home
                     +batter.hbp_Away+pitcher.hbp_Home+batter.so_Away+pitcher.so_Home)*field_multiplier):
                batter.cum_so_L+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
                
        elif(batter_against=='L' and pitcher_against=='R' and batting_side=='Home'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_L+pitcher.singles_R)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_multiplier):
                batter.cum_singles_L+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away)*field_multiplier):
                batter.cum_triples_L+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R
                      +batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R)*hand_multiplier
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                    +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away)*field_multiplier):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                     +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away)*field_multiplier):
                batter.cum_walks_L+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home
                     +pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home
                     +pitcher.walks_Away+batter.hbp_Home+pitcher.hbp_Away)*field_multiplier):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R+batter.so_L+pitcher.so_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away
                     +batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away
                     +batter.hbp_Home+pitcher.hbp_Away+batter.so_Home+pitcher.so_Away)*field_multiplier):
                batter.cum_so_L+=1                
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
                
        elif(batter_against=='L' and pitcher_against=='R' and batting_side=='Away'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_L+pitcher.singles_R)*hand_multiplier + (batter.singles_Away+pitcher.singles_Home)*field_multiplier):
                batter.cum_singles_L+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home)*field_multiplier):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home)*field_multiplier):
                batter.cum_triples_L+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R
                      +batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R)*hand_multiplier
                    + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                    +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home)*field_multiplier):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                     +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home)*field_multiplier):
                batter.cum_walks_L+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away
                     +pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away
                     +pitcher.walks_Home+batter.hbp_Away+pitcher.hbp_Home)*field_multiplier):
                batter.cum_hbp_L+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<(batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R
                     +batter.home_runs_L+pitcher.home_runs_R+batter.walks_L+pitcher.walks_R+batter.hbp_L+pitcher.hbp_R+batter.so_L+pitcher.so_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home
                     +batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home
                     +batter.hbp_Away+pitcher.hbp_Home+batter.so_Away+pitcher.so_Home)*field_multiplier):
                batter.cum_so_L+=1                
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
                
        elif(batter_against=='R' and pitcher_against=='L' and batting_side=='Home'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_R+pitcher.singles_L)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_multiplier):
                batter.cum_singles_R+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away)*field_multiplier):
                batter.cum_triples_R+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L
                      +batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L)*hand_multiplier
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                    +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away)*field_multiplier):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                     +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away)*field_multiplier):
                batter.cum_walks_R+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home
                     +pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home
                     +pitcher.walks_Away+batter.hbp_Home+pitcher.hbp_Away)*field_multiplier):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L+batter.so_R+pitcher.so_L)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away
                     +batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away
                     +batter.hbp_Home+pitcher.hbp_Away+batter.so_Home+pitcher.so_Away)*field_multiplier):
                batter.cum_so_R+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
                
        elif(batter_against=='R' and pitcher_against=='L' and batting_side=='Away'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_R+pitcher.singles_L)*hand_multiplier + (batter.singles_Away+pitcher.singles_Home)*field_multiplier):
                batter.cum_singles_R+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home)*field_multiplier):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home)*field_multiplier):
                batter.cum_triples_R+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L
                      +batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L)*hand_multiplier
                    + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                    +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home)*field_multiplier):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                     +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home)*field_multiplier):
                batter.cum_walks_R+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away
                     +pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away
                     +pitcher.walks_Home+batter.hbp_Away+pitcher.hbp_Home)*field_multiplier):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_L+=1
                return "hbp"
            elif(num<(batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L
                     +batter.home_runs_R+pitcher.home_runs_L+batter.walks_R+pitcher.walks_L+batter.hbp_R+pitcher.hbp_L+batter.so_R+pitcher.so_L)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home
                     +batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home
                     +batter.hbp_Away+pitcher.hbp_Home+batter.so_Away+pitcher.so_Home)*field_multiplier):
                batter.cum_so_R+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:

                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
                
        elif(batter_against=='R' and pitcher_against=='R' and batting_side=='Home'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_R+pitcher.singles_R)*hand_multiplier + (batter.singles_Home+pitcher.singles_Away)*field_multiplier):
                batter.cum_singles_R+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away)*field_multiplier):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away)*field_multiplier):
                batter.cum_triples_R+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R
                      +batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R)*hand_multiplier
                    + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                    +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away)*field_multiplier):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away
                     +batter.triples_Home+pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away)*field_multiplier):
                batter.cum_walks_R+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home
                     +pitcher.triples_Away+batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home
                     +pitcher.walks_Away+batter.hbp_Home+pitcher.hbp_Away)*field_multiplier):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R+batter.so_R+pitcher.so_R)*hand_multiplier
                     + (batter.singles_Home+pitcher.singles_Away+batter.doubles_Home+pitcher.doubles_Away+batter.triples_Home+pitcher.triples_Away
                     +batter.home_runs_Home+pitcher.home_runs_Away+batter.walks_Home+pitcher.walks_Away
                     +batter.hbp_Home+pitcher.hbp_Away+batter.so_Home+pitcher.so_Away)*field_multiplier):
                batter.cum_so_R+=1                
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
              
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
                
        elif(batter_against=='R' and pitcher_against=='R' and batting_side=='Away'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            
            if(num<(batter.singles_R+pitcher.singles_R)*hand_multiplier + (batter.singles_Away+pitcher.singles_Home)*field_multiplier):
                batter.cum_singles_R+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home)*field_multiplier):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home)*field_multiplier):
                batter.cum_triples_R+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R
                      +batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R)*hand_multiplier
                    + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                    +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home)*field_multiplier):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home
                     +batter.triples_Away+pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home)*field_multiplier):
                batter.cum_walks_R+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away
                     +pitcher.triples_Home+batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away
                     +pitcher.walks_Home+batter.hbp_Away+pitcher.hbp_Home)*field_multiplier):
                batter.cum_hbp_R+=1
                pitcher.cum_hbp_R+=1
                return "hbp"
            elif(num<(batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R
                     +batter.home_runs_R+pitcher.home_runs_R+batter.walks_R+pitcher.walks_R+batter.hbp_R+pitcher.hbp_R+batter.so_R+pitcher.so_R)*hand_multiplier
                     + (batter.singles_Away+pitcher.singles_Home+batter.doubles_Away+pitcher.doubles_Home+batter.triples_Away+pitcher.triples_Home
                     +batter.home_runs_Away+pitcher.home_runs_Home+batter.walks_Away+pitcher.walks_Home
                     +batter.hbp_Away+pitcher.hbp_Home+batter.so_Away+pitcher.so_Home)*field_multiplier):
                batter.cum_so_R+=1                
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 
                #if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                if(outs<2 and runners[0]!=None and num>1.9):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
                            
    def move_runners(self, team, batter, pitcher, outs, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            batter who is hitting
            pitcher: pitcher pitching to team
            outs: number of outs
            result: str of result for batter (ie "walk")
        """
        made_out = False

        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        if(result=='single'):            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                team.runners[2] = None
                
            if(team.runners[1]!=None): 
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                
                #if(num<((batter.bsr*2)+45+(outs*10))):
                if(num<50):
                    new_num = random.uniform(0,100)
                    if(new_num<3):
                        if(pitcher_against=='L'):
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[1].cum_runs+=1
                        team.runners_dict[team.runners[1]].cum_runs+=1
                        team.runners_dict[team.runners[1]].current_runs+=1
                        batter.cum_rbi+=1
                        team.runners[3]+=1
                else:
                    team.runners[2] = team.runners[1]
                    
                team.runners[1] = None
                
            if(team.runners[0]!=None and team.runners[2]==None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically

                #if(num<((batter.bsr*3)+30)):
                if(num<30):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[2] = team.runners[0] 
                else:
                    team.runners[1] = team.runners[0]
            else:
                team.runners[1] = team.runners[0]
                
            team.runners[0] = batter
            team.runners_dict.update({batter: pitcher})

        elif(result=='double'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            
            if(team.runners[0]!=None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                #if(num<((batter.bsr*4)+45)):
                if(num<50):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[0].cum_runs+=1
                        team.runners_dict[team.runners[0]].cum_runs+=1
                        team.runners_dict[team.runners[0]].current_runs+=1
                        batter.cum_rbi+=1
                        team.runners[3]+=1 
                else:
                    team.runners[2] = team.runners[0]
                
            team.runners[1] = batter
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='triple'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = batter
            team.runners[1] = None
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='home_run'):
            batter.cum_runs+=1
            pitcher.cum_runs+=1
            pitcher.current_runs+=1
            batter.cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=='walk' or result=='hbp' or result=='sh'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            team.runners_dict.update({batter: pitcher})
            
            if(temp_0!=None and temp_1!=None and temp_2!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            
            team.runners[0] = batter
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
            if(result=='sh'):   # If sacrifice is hit, batter doesn't reach base
                team.runners[0] = None
                                          
        elif(result=='gdp'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            rand_num = random.uniform(0,1)

            if(temp_1==None and temp_2==None):    # If only runner on first
                team.runners[0] = None
            elif(temp_1!=None and temp_2==None):    # If runners on first and second
                if(rand_num<.02):   # Outs made at first and third
                    team.runners[1] = temp_0
                    team.runners[0] = None
                elif(rand_num<.08):   # Outs made at third and second
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                else:
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1==None and temp_2!=None):    # If runners on first and third
                if(rand_num<.01):   # Batter goes to first
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                elif(rand_num<.02):   # Batter goes to second
                    team.runners[0] = None
                    team.runners[1] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                else: # Runner scores, bases empty
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[2] = None
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1!=None and temp_2!=None):    # If bases loaded
                if(rand_num<.6):   # Outs made at second and first
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[0] = None
                    team.runners[1] = None
                    team.runners[2] = temp_1
                elif(rand_num<.9):   # Outs made at home and first
                    team.runners[1] = temp_0
                    team.runners[2] = temp_1
                    team.runners[0] = None
                else:    # Outs made at third and home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                                        
        elif(result=='out'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            if(temp_0!=None and temp_1==None and temp_2==None):
                rand_num = random.uniform(0,1)
                
                #if(rand_num<(.3+batter.bsr/15)):   # If batter hits ground ball and reaches on fielder's choice
                if(rand_num<0.3):    
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.4):    # If batter hits ground ball and is out but runner on first moves to second
                    team.runners[1] = temp_1
                    team.runners[0] = None
            elif(temp_0==None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.3+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third or grounder moving runner
                if(rand_num<0.3): 
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                if(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                elif(rand_num<.4):   # If batter hits ground ball scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
            elif(temp_0!=None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.1+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third
                if(rand_num<0.1): 
                    team.runners[2] = temp_1
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, guy on second goes to third, guy on first is out, batter goes to first
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.35):    # If batter hits ball, guy on second goes to third, guy on first goes to second, batter makes out
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = None
            elif(temp_0!=None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                if(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.05):
                        team.runners[1] = temp_0
                    
                    team.runners[2] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                #elif(rand_num<.35+batter.bsr/15):    # If batter hits ball, guy on third scores, guy on first out, batter ends up on first
                elif(rand_num<0.35): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                if(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runner on second advances
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.33):    # If batter hits ground ball, out made at home, runner on second stays
                    team.runners[2] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0!=None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                #if(rand_num<(.27+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                if(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    batter.cum_sf+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    if(rand_num<.06):
                        team.runners[1] = temp_0

                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runners advance
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                #elif(rand_num<.4+batter.bsr/15):    # If batter hits ground ball, out made at home, runner on second stays
                elif(rand_num<0.3): 
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})

        if(team.runners[3]>0):
            #print("adding runs", team.runners[3])
            team.runs+=team.runners[3]
            team.runners[3] = 0
            
        if(made_out):
            return 'out'

        #print("updated runners: " + str(team.runners))
        
    def print_result(self):
        """Printed score of a game
        """ 
        print("final score: " + self.away_team.city + " " + str(self.away_team.runs) + " " 
              + self.home_team.city + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams
        """ 
        print(str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played
        """ 
        self.away_team.restart()
        self.away_team.game_no+=1
        self.away_team.rotation_index = (self.away_team.game_no-1)%5
        self.home_team.restart() 
        self.home_team.game_no+=1
        self.home_team.rotation_index = (self.home_team.game_no-1)%5
        inning = 1
        game_over = False
        prin = False
        
         
        while(inning<self.innings+1 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                if side==self.away_team:
                    batting_team='Away'
                else:
                    batting_team = 'Home'
                    
                if(inning==self.innings and side==self.home_team and self.home_team.runs>self.away_team.runs):
                    game_over = True
                    break
     
                outs = 0

                if(inning==1 and side==self.away_team):
                    # Determining starting pitchers
                    home_pitcher = self.home_team.starter
                    away_pitcher = self.away_team.starter
                    self.home_team.starter.cum_G+=1
                    self.home_team.starter.cum_GS+=1
                    self.away_team.starter.cum_G+=1
                    self.away_team.starter.cum_GS+=1
                    self.home_team.current_pitcher_list.append(home_pitcher)
                    self.home_team.current_pitcher = home_pitcher
                    self.away_team.current_pitcher_list.append(away_pitcher)
                    self.away_team.current_pitcher = away_pitcher
                    
                    # Making sure interleague lineups are right
                    if self.away_team.league=='AL' and self.home_team.league=='NL' and self.away_team.lineup[8].name=='extra8':
                        self.away_team.batters.remove(self.away_team.lineup[8])
                        self.away_team.lineup[8] = Batter_new(self.away_team.city, 'pitcher', -1, 'R', .06, .01, -.05, -.03, -.03, 0, .5, .06, .01, 
                                                              -.05, -.03, -.03, 0, .5, .06, .01, -.05, -.03, -.03, 0, .5, .06, .01, 
                                                              -.05, -.03, -.03, 0, .5, 0, 0)

                    
                        self.away_team.batters.append(self.away_team.lineup[8])
                        
                    for num in range(9):
                        self.away_team.game_lineup[num].cum_G+=1
                        self.home_team.game_lineup[num].cum_G+=1

                while(outs<3):
                    # Choosing a pitcher for a team based on game factors
                    if(side==self.away_team):
                        current_pitcher = self.home_team.current_pitcher
                        pitcher = self.home_team.choose_pitcher(outs, inning, self.innings, side.runs, self.home_team.runs, side.game_lineup, 
                                                                side.batting_index, self.home_team.game_lineup, self.home_team.batting_index)
                        pitcher.finished_half = False
                        
                        if(current_pitcher!=pitcher and self.home_team.game_lineup[8].name=='pitcher'):   # If starting pitcher gets taken out of game and lineup, put bench player in to bat
                            self.home_team.game_lineup[8] = self.home_team.bench[random.randint(0, len(self.home_team.bench)-1)]
                            self.home_team.game_lineup[8].cum_G+=1
                            
                    elif(side==self.home_team):
                        current_pitcher = self.away_team.current_pitcher
                        pitcher = self.away_team.choose_pitcher(outs, inning, self.innings, side.runs, self.away_team.runs, side.game_lineup, 
                                                                side.batting_index, self.away_team.game_lineup, self.away_team.batting_index)
                        pitcher.finished_half = False
                        
                        if(current_pitcher!=pitcher and self.away_team.game_lineup[8].name=='pitcher'):   # If starting pitcher gets taken out of game and lineup, put bench player in to bat
                            self.away_team.game_lineup[8] = self.away_team.bench[random.randint(0, len(self.away_team.bench)-1)]
                            self.away_team.game_lineup[8].cum_G+=1
                            
                    # Determining whether stolen base should be attempted
                    stolen_base_result = self.get_sb_wp_result(side.runners, pitcher, side, outs, random.uniform(0,1))
                    
                    if(stolen_base_result=='out'):   # If failed stolen base, out is recorded
                        pitcher.current_outs+=1
                        outs+=1
                                
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.runners_dict = {}
                            break
                    
                    # Determining outcome of at-bat
                    rand_num = random.uniform(0,2)
                    result = self.get_result(side.game_lineup[side.batting_index], pitcher, outs, side.runners, batting_team, rand_num)
                    
                    temp_outs = outs
                    if(result=='out' or result=='k' or result=='sf'):
                        temp_outs+=1
                    elif(result=='gdp'):
                        temp_outs+=1
                    
                   # print("inning: " + str(inning) + '\tpitcher: ' + pitcher.name + '\t\tbatter: ' + side.game_lineup[side.batting_index].name + '\t\tresult: ' + result 
                    #      + '\touts: ' + str(temp_outs)  + '\t' + self.away_team.city + ': ' + str(self.away_team.runs) + '\t' + self.home_team.city + ': ' + str(self.home_team.runs))
                        
                    if(result=='out' or result=='k' or result=='sf'):    # Batter made an out
                        pitcher.current_outs+=1
                        outs+=1

                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.game_lineup[side.batting_index], pitcher, outs, result)
                            
                    elif(result=='gdp'):    # Batter made an out
                        pitcher.current_outs+=2
                        outs+=2

                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.game_lineup[side.batting_index], pitcher, outs, result)
                            
                    else:    # Batter reached, runners are moved
                        running_result = self.move_runners(side, side.game_lineup[side.batting_index], pitcher, outs, result)
                        if(running_result=='out'):    # Baserunner made an out
                            pitcher.current_outs+=1
                            outs+=1
                            
                            if(outs>2):   # If outs = 3, end of inning
                                pitcher.finished_half = True
                                side.batting_index+=1    # Gettting next batter in lineup
                                side.batting_index = side.batting_index%9
                                side.runners_dict = {}
                                break

                        if(inning>9 and self.away_team.runs<self.home_team.runs):
                            game_over = True

                            if(game_over):    # If game is over, escape game loop
                                break
                        
                    side.batting_index+=1    # Gettting next batter in lineup
                    side.batting_index = side.batting_index%9
                    
                if(game_over):    # If game is over, escape game loop
                    break
                        
                side.reset_runners()    # Runners are reset after half inning
            

            if(game_over):    # If game is over, escape game loop
                break   
                
            inning+=1
        
        # After game, updating team records and number of cumulative runs
        
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             

In [ ]:
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
  
def writing_data(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)
    
def date_formatter(row):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    if row.date == '':
        return(row.date)
    else:
        split_date = row.date.split('-')
        day = split_date[0]
        day = day.zfill(2)
        month = strptime(split_date[1],'%b').tm_mon
        date = str(month) +  '/' + day + '/' + '2020'
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        date = date.strftime('%m/%d/%Y')
        
    return date

def fangraphs_daily_lineups(lineup_dict={}, lineup_id_dict={}, rosters={}, team_hand_dict={}):
    url = 'https://www.fangraphs.com/livescoreboard.aspx?'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    
    schedule_dict = {}
    player_dict = {}
    new_lineups = []

    header = soup.find('tr', attrs={'class': 'win-board__header'})
    
    file_name = 'Baseball Picks.csv'
    current_spreadsheet = getting_spreadsheet(file_name) # Retrieving Spreadsheet
    current_spreadsheet['Date'] = current_spreadsheet['Date'].apply(lambda date: spreadsheet_date_formatter(date))
    current_spreadsheet['Date'] = current_spreadsheet['Date'].replace('', np.nan).ffill()
    current_spreadsheet['Time'] = current_spreadsheet['Time'].replace('', np.nan).ffill()
    
    # Finding new games and team lineups
    for games in header.next_siblings:
        for matchup in games.findAll('td', style=True, align=False):
            away_team = None
            home_team = None
            
            game_time= matchup.get_text().split(' ET')[0][-5:].strip().split(':')[0]   # Time of game (hour it starts in army time eastern)
            
            if game_time not in schedule_dict.keys() and game_time!='':
                schedule_dict[game_time] = []
            
            for num, team_names in enumerate(matchup.findAll('a')): # Getting team names
                
                if(num==0):
                    if team_names.get_text().strip().capitalize() in league.keys():
                        away_team = team_names.get_text().strip().capitalize()
                    else:
                        away_team = nickname_to_city_dict[team_names.get_text().strip().lower()]
 
                    for teams in list(chain.from_iterable(schedule_dict.values())):     # Finding teams that are in the second leg of a double header
                        if(away_team in [teams[0], teams[1]] or len(current_spreadsheet.loc[(current_spreadsheet['Team']==away_team) 
                                                                   & (current_spreadsheet['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))
                                                                   & (current_spreadsheet['Time']<int(game_time))])>0):                    
                            away_team = away_team + '2'
                elif(num==1):

                    if team_names.get_text().strip().capitalize() in league.keys():
                        home_team = team_names.get_text().strip().capitalize()
                    else:
                        home_team = nickname_to_city_dict[team_names.get_text().strip().lower()]

                    for teams in list(chain.from_iterable(schedule_dict.values())):     # Finding teams that are in the second leg of a double header   
                        if(home_team in [teams[0], teams[1]] or len(current_spreadsheet.loc[(current_spreadsheet['Team']==home_team) 
                                                                   & (current_spreadsheet['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))
                                                                   & (current_spreadsheet['Time']<int(game_time))])>0):                    
                            home_team = home_team + '2'
                        
            if(away_team!=None and home_team!=None):   # Adding matchup to schedule dict
                schedule_dict[game_time].append((away_team, home_team))            
            
            away_lineup_dict = {}
            away_lineup_id_dict = {}
            away_lineup = []
            away_lineup_id = []
            away_starter = None
            away_starter_id = None
            away_opener = None
            away_opener_id = None
            away_probable = None
            away_probable_id = None
            home_lineup_dict = {}
            home_lineup_id_dict = {}
            home_lineup = []
            home_lineup_id = []
            home_starter = None
            home_starter_id = None
            home_opener = None
            home_opener_id = None
            home_probable = None
            home_probable_id = None

            lineup = matchup.find('table', attrs={'class', 'lineup'})
            if(lineup!=None):
                for num, position in enumerate(lineup.findAll('td', attrs={'align': 'left'})):

                    if(num%2==0):
                        if position.get_text()[0:2] == 'SP':
                            away_starter = position.a.get_text().strip()
                            away_starter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            #player_dict[away_starter] = away_starter_id
                        elif position.get_text()[0:2] == 'OP':
                            for index, player in enumerate(position.findAll('a')):
                                if index==0:
                                    away_opener = player.get_text().strip()
                                    away_opener_id = player['href'].split('playerid=')[1].split('&')[0]
                                    #player_dict[away_opener] = away_opener_id
                                elif index==1:
                                    away_probable = player.get_text().strip()
                                    away_probable_id = player['href'].split('playerid=')[1].split('&')[0]
                                    #player_dict[away_probable] = away_probable_id
                                else:
                                    away_batter = player.get_text().strip()
                                    away_lineup.append(away_batter)
                                    away_batter_id = player['href'].split('playerid=')[1].split('&')[0]
                                    away_lineup_id.append(away_batter_id)
                                    #player_dict[away_batter] = away_batter_id
                        elif position.get_text()[0:2] not in ('SP', 'OP'):
                            for player in position.findAll('a'):
                                away_batter = player.get_text().strip()
                                away_lineup.append(away_batter)
                                away_batter_id = player['href'].split('playerid=')[1].split('&')[0]
                                away_lineup_id.append(away_batter_id)
                                #player_dict[away_batter] = away_batter_id

                        away_lineup_dict['starter'] = away_starter
                        away_lineup_dict['opener'] = away_opener
                        away_lineup_dict['probable'] = away_probable
                        away_lineup_dict['lineup'] = away_lineup
                        
                        away_lineup_id_dict['starter'] = away_starter_id
                        away_lineup_id_dict['opener'] = away_opener_id
                        away_lineup_id_dict['probable'] = away_probable_id
                        away_lineup_id_dict['lineup'] = away_lineup_id

                    elif(num%2==1):
                        if position.get_text()[0:2] == 'SP':
                            home_starter = position.a.get_text().strip()
                            home_starter_id = position.a['href'].split('playerid=')[1].split('&')[0]
                            #player_dict[home_starter] = home_starter_id
                        elif position.get_text()[0:2] == 'OP':
                            for index, player in enumerate(position.findAll('a')):
                                if index==0:
                                    home_opener = player.get_text().strip()
                                    home_opener_id = player['href'].split('playerid=')[1].split('&')[0]
                                    #player_dict[home_opener] = home_opener_id
                                elif index==1:
                                    home_probable = player.get_text().strip()
                                    home_probable_id = player['href'].split('playerid=')[1].split('&')[0]
                                    #player_dict[home_probable] = home_probable_id
                                else:
                                    home_batter = player.get_text().strip()
                                    home_lineup.append(home_batter)
                                    home_batter_id = player['href'].split('playerid=')[1].split('&')[0]
                                    home_lineup_id.append(home_batter_id)
                                    #player_dict[home_batter] = home_batter_id
                                    
                        elif position.get_text()[0:2] not in ('SP', 'OP'):
                            for player in position.findAll('a'):
                                home_batter = player.get_text().strip()
                                home_lineup.append(home_batter)
                                home_batter_id = player['href'].split('playerid=')[1].split('&')[0]
                                home_lineup_id.append(home_batter_id)
                                #player_dict[home_batter] = home_batter_id

                        home_lineup_dict['starter'] = home_starter
                        home_lineup_dict['opener'] = home_opener
                        home_lineup_dict['lineup'] = home_lineup  
                        home_lineup_dict['probable'] = home_probable
                        
                        home_lineup_id_dict['starter'] = home_starter_id
                        home_lineup_id_dict['opener'] = home_opener_id
                        home_lineup_id_dict['lineup'] = home_lineup_id  
                        home_lineup_id_dict['probable'] = home_probable_id

                  
                #if((len(lineup_dict[away_team]['lineup'])>0 or len(away_lineup_dict['lineup']>0)     # Only update for games where lineups exist
                    #and (len(lineup_dict[away_team]['lineup'])>0 or len(away_lineup_dict['lineup']>0))):

                if(away_team not in lineup_dict.keys() or (lineup_dict[away_team]!=away_lineup_dict 
                   and len(current_spreadsheet.loc[(current_spreadsheet['Team']==away_team) & (current_spreadsheet['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))
                        & (current_spreadsheet['Time']==game_time)])==0) or home_team not in lineup_dict.keys() or (lineup_dict[home_team]!=home_lineup_dict
                   and len(current_spreadsheet.loc[(current_spreadsheet['Team']==home_team) & (current_spreadsheet['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))
                            & (current_spreadsheet['Time']==game_time)])==0)):
                    new_lineups.append(away_team)
                    new_lineups.append(home_team)
                    
                #if lineup already exists and new lineup is empty, do not replace existing lineup
                if not (away_team in lineup_dict.keys() and len(lineup_dict[away_team]['lineup'])>0  and len(away_lineup_dict['lineup'])==0):   
                #if away_team=='Atlanta':
                    lineup_dict[away_team] = away_lineup_dict
                    lineup_id_dict[away_team] = away_lineup_id_dict
                    rosters[away_team], team_hand_dict[away_team] = active_roster(away_team)

                if not (home_team in lineup_dict.keys() and len(lineup_dict[home_team]['lineup'])>0  and len(home_lineup_dict['lineup'])==0): 
                #if home_team=='Philadelphia':
                    lineup_dict[home_team] = home_lineup_dict
                    lineup_id_dict[home_team] = home_lineup_id_dict
                    rosters[home_team], team_hand_dict[home_team] = active_roster(home_team)

    return lineup_dict, lineup_id_dict, rosters, team_hand_dict, schedule_dict, new_lineups

def create_teams_new(batter_stats, pitcher_stats, lineup_dict={}, lineup_id_dict={}, team_dict={}, mlb_rosters={}, rosters={}, team_hand_dict={}, new_lineups=[], soup_dict={}):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            batter_stats: df of batter stats
            pitcher_stats: df of pitcher stats
            lineup_dict: dict of team lineups and starting pitchers
            rosters: active rosters of players for each team
        Returns:
            team_dict: dict with city of team as key and Team instance as value
    """
    global id_arms_dict, id_bats_dict, batter_stats_df, pitcher_stats_df, team_num_dict
    
    batter_positions = ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']
    pitcher_positions = ['SP', 'RP']
    #url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='

    for team_name in lineup_dict.keys():    # Get list of url's for teams playing
        #url_list.append(url_start+str(team_num_dict[name])) 
        if ((team_name not in new_lineups and team_name in team_dict.keys()) or len(lineup_dict[team_name]['lineup'])==0):
            continue
        
        name = team_name.split('2')[0]
        
        if team_name[-1]=='2':   # If team is from second leg of double header, no need to update roster information
            team_dict.update({team_name: Team_new(name, league[name], batter_df, pitcher_df, rosters[name], 
                                                  batter_stats, pitcher_stats, lineup_dict[name], lineup_id_dict[name])})
            continue
        
        if team_name not in soup_dict.keys():
            url = url_start+str(team_num_dict[name])
            headers = {'User-Agent': header_name}
            source = requests.get(url, headers=headers)
            soup = BeautifulSoup(source.content, 'html.parser')
            soup_dict[team_name] = soup
        else:
            soup = soup_dict[team_name]
          
        positional_values_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()
        pos_dict = {}
        
        if team_name not in rosters.keys():
            rosters[team_name] = {}
        
        for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP']}):#, '#ALL']}):
            pos = a_tag.get_text()
            pos_dict[pos] = {}
            rosters[team_name][pos] = []

            table = a_tag.find_next('table')

            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                if not row.a:
                    continue
                    
                info = row.a['href']
                elements = info.split('=')
                player_ID = elements[1].split('&')[0]

                if not player_ID.isdigit():
                    continue

                player_name = unidecode.unidecode(row.a.get_text().strip())   # Removing accent marks
                pos_dict[pos][player_ID] = player_name
                
                if pos in batter_positions:
                    if player_name in mlb_rosters[team_name]['Batters']:
                        rosters[team_name][pos].append(player_ID)
                elif pos in pitcher_positions:
                    if player_name in mlb_rosters[team_name]['Pitchers']:
                        rosters[team_name][pos].append(player_ID)
                
                if 'Closer' in mlb_rosters[team_name].keys() and player_name==mlb_rosters[team_name]['Closer']:
                    rosters[team_name]['Closer'] = player_ID
        
        for position, player_info in pos_dict.items():
            for player_ID, player_name in player_info.items():
                
                if position not in rosters[team_name].keys() or player_ID not in rosters[team_name][position]:
                    continue

                if position in batter_positions:   # Creating dataframe for batters

                    if(not batter_df.empty and player_ID in batter_df['playerID'].values.tolist()):  # Player already in batter_df at different position
                        batter_df.loc[batter_df['playerID']==player_ID, position] = 1
                    else:

                        batter_headers = ['Name', 'playerID', 'Team', 'Bats', 'PA_L', 'AB_L',
                                           'singles_L', 'doubles_L', 'triples_L', 'HR_L', 
                                           'BB_L', 'SO_L', 'HBP_L', 'SB_L', 'CS_L', 
                                           'PA_R', 'AB_R', 'singles_R', 'doubles_R', 
                                           'triples_R', 'HR_R', 'BB_R', 'SO_R', 'HBP_R', 
                                           'SB_R', 'CS_R', 'PA_Home', 'AB_Home',
                                           'singles_Home', 'doubles_Home', 'triples_Home', 'HR_Home', 
                                           'BB_Home', 'SO_Home', 'HBP_Home', 'SB_Home', 'CS_Home', 
                                           'PA_Away', 'AB_Away', 'singles_Away', 'doubles_Away', 
                                           'triples_Away', 'HR_Away', 'BB_Away', 'SO_Away', 'HBP_Away', 
                                           'SB_Away', 'CS_Away', 'C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'SP', 'RP']

                        player_stats = batter_stats.loc[batter_stats['playerId']==int(player_ID)]

                        if player_stats.empty:  # Change this after finding handedness of all players
                            player_stats = new_player_stats(soup_dict[team_name], player_name, player_ID, position, team_hand_dict[team_name][player_name], team_name)

                        if(player_stats['Bats'].tolist()[0]=='L'):
                            PA_L = 50
                            AB_L = 45
                            singles_L = 7
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 2
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 12
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 12
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 4
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 18
                            SB_R = 1
                            CS_R = 0
                        elif(player_stats['Bats'].tolist()[0]=='R'):
                            PA_L = 50
                            AB_L = 45
                            singles_L = 8
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 3
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 10
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 10
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 3
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 20
                            SB_R = 1
                            CS_R = 0
                        else:
                            PA_L = 50
                            AB_L = 45
                            singles_L = 8
                            doubles_L = 2
                            triples_L = 0
                            HR_L = 3
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 10
                            SB_L = 1
                            CS_L = 0

                            PA_R = 80
                            AB_R = 72
                            singles_R = 12
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 4
                            BB_R = 7
                            HBP_R = 1
                            SO_R = 18
                            SB_R = 1
                            CS_R = 0

                        PA_Home = 80
                        AB_Home = 72
                        singles_Home = 12
                        doubles_Home = 4
                        triples_Home = 0
                        HR_Home = 4
                        BB_Home = 7
                        HBP_Home = 1
                        SO_Home = 18
                        SB_Home = 1
                        CS_Home = 0

                        PA_Away = 80
                        AB_Away = 72
                        singles_Away = 12
                        doubles_Away = 4
                        triples_Away = 0
                        HR_Away = 4
                        BB_Away = 7
                        HBP_Away = 1
                        SO_Away = 18
                        SB_Away = 1
                        CS_Away = 0

                        seasons_list = sorted(player_stats['Season'].to_list())
                        for year in seasons_list:
                            multiplier = (1-(max(seasons_list)-year)/len(seasons_list))

                            PA_L += player_stats.loc[player_stats['Season']==year]['PA_L'].values[0]*multiplier
                            AB_L += player_stats.loc[player_stats['Season']==year]['AB_L'].values[0]*multiplier
                            singles_L += player_stats.loc[player_stats['Season']==year]['singles_L'].values[0]*multiplier
                            doubles_L += player_stats.loc[player_stats['Season']==year]['doubles_L'].values[0]*multiplier
                            triples_L += player_stats.loc[player_stats['Season']==year]['triples_L'].values[0]*multiplier
                            HR_L += player_stats.loc[player_stats['Season']==year]['HR_L'].values[0]*multiplier
                            BB_L += player_stats.loc[player_stats['Season']==year]['BB_L'].values[0]*multiplier
                            HBP_L += player_stats.loc[player_stats['Season']==year]['HBP_L'].values[0]*multiplier
                            SO_L += player_stats.loc[player_stats['Season']==year]['SO_L'].values[0]*multiplier
                            #SB_L += player_stats.loc[player_stats['Season']==year]['SB_L'].values[0]*multiplier
                            #CS_L += player_stats.loc[player_stats['Season']==year]['CS_L'].values[0]*multiplier

                            PA_R += player_stats.loc[player_stats['Season']==year]['PA_R'].values[0]*multiplier
                            AB_R += player_stats.loc[player_stats['Season']==year]['AB_R'].values[0]*multiplier
                            singles_R += player_stats.loc[player_stats['Season']==year]['singles_R'].values[0]*multiplier
                            doubles_R += player_stats.loc[player_stats['Season']==year]['doubles_R'].values[0]*multiplier
                            triples_R += player_stats.loc[player_stats['Season']==year]['triples_R'].values[0]*multiplier
                            HR_R += player_stats.loc[player_stats['Season']==year]['HR_R'].values[0]*multiplier
                            BB_R += player_stats.loc[player_stats['Season']==year]['BB_R'].values[0]*multiplier
                            HBP_R += player_stats.loc[player_stats['Season']==year]['HBP_R'].values[0]*multiplier
                            SO_R += player_stats.loc[player_stats['Season']==year]['SO_R'].values[0]*multiplier
                            #SB_R += player_stats.loc[player_stats['Season']==year]['SB_R'].values[0]*multiplier
                            #CS_R += player_stats.loc[player_stats['Season']==year]['CS_R'].values[0]*multiplier

                            PA_Home += player_stats.loc[player_stats['Season']==year]['PA_Home'].values[0]*multiplier
                            AB_Home += player_stats.loc[player_stats['Season']==year]['AB_Home'].values[0]*multiplier
                            singles_Home += player_stats.loc[player_stats['Season']==year]['singles_Home'].values[0]*multiplier
                            doubles_Home += player_stats.loc[player_stats['Season']==year]['doubles_Home'].values[0]*multiplier
                            triples_Home += player_stats.loc[player_stats['Season']==year]['triples_Home'].values[0]*multiplier
                            HR_Home += player_stats.loc[player_stats['Season']==year]['HR_Home'].values[0]*multiplier
                            BB_Home += player_stats.loc[player_stats['Season']==year]['BB_Home'].values[0]*multiplier
                            HBP_Home += player_stats.loc[player_stats['Season']==year]['HBP_Home'].values[0]*multiplier
                            SO_Home += player_stats.loc[player_stats['Season']==year]['SO_Home'].values[0]*multiplier
                            #SB_Home += player_stats.loc[player_stats['Season']==year]['SB_Home'].values[0]*multiplier
                            #CS_Home += player_stats.loc[player_stats['Season']==year]['CS_Home'].values[0]*multiplier

                            PA_Away += player_stats.loc[player_stats['Season']==year]['PA_Away'].values[0]*multiplier
                            AB_Away += player_stats.loc[player_stats['Season']==year]['AB_Away'].values[0]*multiplier
                            singles_Away += player_stats.loc[player_stats['Season']==year]['singles_Away'].values[0]*multiplier
                            doubles_Away += player_stats.loc[player_stats['Season']==year]['doubles_Away'].values[0]*multiplier
                            triples_Away += player_stats.loc[player_stats['Season']==year]['triples_Away'].values[0]*multiplier
                            HR_Away += player_stats.loc[player_stats['Season']==year]['HR_Away'].values[0]*multiplier
                            BB_Away += player_stats.loc[player_stats['Season']==year]['BB_Away'].values[0]*multiplier
                            HBP_Away += player_stats.loc[player_stats['Season']==year]['HBP_Away'].values[0]*multiplier
                            SO_Away += player_stats.loc[player_stats['Season']==year]['SO_Away'].values[0]*multiplier
                            #SB_Away += player_stats.loc[player_stats['Season']==year]['SB_Away'].values[0]*multiplier
                            #CS_Away += player_stats.loc[player_stats['Season']==year]['CS_Away'].values[0]*multiplier


                        stats = pd.Series({'Name': player_name, 'playerID': player_ID, 'Team': team_name, 
                                           'Bats': player_stats['Bats'].tolist()[0], 'PA_L': PA_L, 'AB_L': AB_L,
                                           'singles_L': singles_L, 'doubles_L': doubles_L, 'triples_L': triples_L, 'HR_L': HR_L, 
                                           'BB_L': BB_L, 'SO_L': SO_L, 'HBP_L': HBP_L, 'SB_L': SB_L, 'CS_L': CS_L, 
                                           'PA_R': PA_R, 'AB_R': AB_R, 'singles_R': singles_R, 'doubles_R': doubles_R, 
                                           'triples_R': triples_R, 'HR_R': HR_R, 'BB_R': BB_R, 'SO_R': SO_R, 'HBP_R': HBP_R, 
                                           'SB_R': SB_R, 'CS_R': CS_R, 'PA_Home': PA_Home, 'AB_Home': AB_Home,
                                           'singles_Home': singles_Home, 'doubles_Home': doubles_Home, 'triples_Home': triples_Home, 'HR_Home': HR_Home, 
                                           'BB_Home': BB_Home, 'SO_Home': SO_Home, 'HBP_Home': HBP_Home, 'SB_Home': SB_Home, 'CS_Home': CS_Home, 
                                           'PA_Away': PA_Away, 'AB_Away': AB_Away, 'singles_Away': singles_Away, 'doubles_Away': doubles_Away, 
                                           'triples_Away': triples_Away, 'HR_Away': HR_Away, 'BB_Away': BB_Away, 'SO_Away': SO_Away, 'HBP_Away': HBP_Away, 
                                           'SB_Away': SB_Away, 'CS_Away': CS_Away, position: 1})



                        if(batter_df.empty):
                            batter_df = pd.DataFrame(columns=batter_headers)

                        batter_df = batter_df.append(stats, ignore_index=True)   # Getting stats from batter stats dataframe
                elif position in pitcher_positions:  # Creating dataframe for pitchers
                    if(not pitcher_df.empty and player_ID in pitcher_df['playerID'].values.tolist()):  # player already in pitcher_df at different position
                        pitcher_df.loc[pitcher_df['playerID']==player_ID, position] = 1
                    else:

                        pitcher_headers = ['Name', 'playerID', 'Team', 'Throws', 'TBF_L',
                                           'singles_L', 'doubles_L', 'triples_L', 'HR_L', 'R_L', 
                                           'ER_L', 'BB_L','HBP_L', 'SO_L', 
                                           'TBF_R',  'singles_R', 'doubles_R', 'triples_R', 'HR_R', 'R_R', 
                                           'ER_R', 'BB_R','HBP_R', 'SO_R', 'TBF_Home',
                                           'singles_Home', 'doubles_Home', 'triples_Home', 'HR_Home', 'R_Home', 
                                           'ER_Home', 'BB_Home','HBP_Home', 'SO_Home', 
                                           'TBF_Away',  'singles_Away', 'doubles_Away', 'triples_Away', 'HR_Away', 'R_Away', 
                                           'ER_Away', 'BB_Away','HBP_Away', 'SO_Away']

                        player_stats = pitcher_stats.loc[pitcher_stats['playerId']==int(player_ID)]

                        if player_stats.empty:  # Change this after finding handedness of all players
                            player_stats = new_player_stats(soup_dict[team_name], player_name, player_ID, position, team_hand_dict[team_name][player_name], team_name)

                        if(player_stats['Throws'].tolist()[0]=='L'):
                            TBF_L = 90
                            singles_L = 14
                            doubles_L = 4
                            triples_L = 0
                            HR_L = 2
                            R_L = 10
                            ER_L = 9
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 25

                            TBF_R = 90
                            singles_R = 15
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 3
                            R_R = 11
                            ER_R = 10
                            BB_R = 5
                            HBP_R = 0
                            SO_R = 20
                        elif(player_stats['Throws'].tolist()[0]=='R'):
                            TBF_L = 90
                            singles_L = 15
                            doubles_L = 4
                            triples_L = 0
                            HR_L = 3
                            R_L = 11
                            ER_L = 10
                            BB_L = 5
                            HBP_L = 0
                            SO_L = 20

                            TBF_R = 90
                            singles_R = 14
                            doubles_R = 4
                            triples_R = 0
                            HR_R = 2
                            R_R = 10
                            ER_R = 9
                            BB_R = 5
                            HBP_R = 0
                            SO_R = 25


                        TBF_Home = 90
                        singles_Home = 14
                        doubles_Home = 4
                        triples_Home = 0
                        HR_Home = 2
                        R_Home = 10
                        ER_Home = 9
                        BB_Home = 5
                        HBP_Home = 0
                        SO_Home = 25

                        TBF_Away = 90
                        singles_Away = 15
                        doubles_Away = 4
                        triples_Away = 0
                        HR_Away = 3
                        R_Away = 11
                        ER_Away = 10
                        BB_Away = 5
                        HBP_Away = 0
                        SO_Away = 20

                        seasons_list = sorted(player_stats['Season'].to_list())
                        for year in seasons_list:
                            
                            multiplier = (1-(max(seasons_list)-year)/len(seasons_list))

                            TBF_L += player_stats.loc[player_stats['Season']==year]['TBF_L'].values[0]*multiplier
                            singles_L += player_stats.loc[player_stats['Season']==year]['singles_L'].values[0]*multiplier
                            doubles_L += player_stats.loc[player_stats['Season']==year]['doubles_L'].values[0]*multiplier
                            triples_L += player_stats.loc[player_stats['Season']==year]['triples_L'].values[0]*multiplier
                            HR_L += player_stats.loc[player_stats['Season']==year]['HR_L'].values[0]*multiplier
                            R_L += player_stats.loc[player_stats['Season']==year]['R_L'].values[0]*multiplier
                            ER_L += player_stats.loc[player_stats['Season']==year]['ER_L'].values[0]*multiplier
                            BB_L += player_stats.loc[player_stats['Season']==year]['BB_L'].values[0]*multiplier
                            HBP_L += player_stats.loc[player_stats['Season']==year]['HBP_L'].values[0]*multiplier
                            SO_L += player_stats.loc[player_stats['Season']==year]['SO_L'].values[0]*multiplier


                            TBF_R += player_stats.loc[player_stats['Season']==year]['TBF_R'].values[0]*multiplier
                            singles_R += player_stats.loc[player_stats['Season']==year]['singles_R'].values[0]*multiplier
                            doubles_R += player_stats.loc[player_stats['Season']==year]['doubles_R'].values[0]*multiplier
                            triples_R += player_stats.loc[player_stats['Season']==year]['triples_R'].values[0]*multiplier
                            HR_R += player_stats.loc[player_stats['Season']==year]['HR_R'].values[0]*multiplier
                            R_R += player_stats.loc[player_stats['Season']==year]['R_R'].values[0]*multiplier
                            ER_R += player_stats.loc[player_stats['Season']==year]['ER_R'].values[0]*multiplier
                            BB_R += player_stats.loc[player_stats['Season']==year]['BB_R'].values[0]*multiplier
                            HBP_R += player_stats.loc[player_stats['Season']==year]['HBP_R'].values[0]*multiplier
                            SO_R += player_stats.loc[player_stats['Season']==year]['SO_R'].values[0]*multiplier

                            TBF_Home += player_stats.loc[player_stats['Season']==year]['TBF_Home'].values[0]*multiplier
                            singles_Home += player_stats.loc[player_stats['Season']==year]['singles_Home'].values[0]*multiplier
                            doubles_Home += player_stats.loc[player_stats['Season']==year]['doubles_Home'].values[0]*multiplier
                            triples_Home += player_stats.loc[player_stats['Season']==year]['triples_Home'].values[0]*multiplier
                            HR_Home += player_stats.loc[player_stats['Season']==year]['HR_Home'].values[0]*multiplier
                            R_Home += player_stats.loc[player_stats['Season']==year]['R_Home'].values[0]*multiplier
                            ER_Home += player_stats.loc[player_stats['Season']==year]['ER_Home'].values[0]*multiplier
                            BB_Home += player_stats.loc[player_stats['Season']==year]['BB_Home'].values[0]*multiplier
                            HBP_Home += player_stats.loc[player_stats['Season']==year]['HBP_Home'].values[0]*multiplier
                            SO_Home += player_stats.loc[player_stats['Season']==year]['SO_Home'].values[0]*multiplier


                            TBF_Away += player_stats.loc[player_stats['Season']==year]['TBF_Away'].values[0]*multiplier
                            singles_Away += player_stats.loc[player_stats['Season']==year]['singles_Away'].values[0]*multiplier
                            doubles_Away += player_stats.loc[player_stats['Season']==year]['doubles_Away'].values[0]*multiplier
                            triples_Away += player_stats.loc[player_stats['Season']==year]['triples_Away'].values[0]*multiplier
                            HR_Away += player_stats.loc[player_stats['Season']==year]['HR_Away'].values[0]*multiplier
                            R_Away += player_stats.loc[player_stats['Season']==year]['R_Away'].values[0]*multiplier
                            ER_Away += player_stats.loc[player_stats['Season']==year]['ER_Away'].values[0]*multiplier
                            BB_Away += player_stats.loc[player_stats['Season']==year]['BB_Away'].values[0]*multiplier
                            HBP_Away += player_stats.loc[player_stats['Season']==year]['HBP_Away'].values[0]*multiplier
                            SO_Away += player_stats.loc[player_stats['Season']==year]['SO_Away'].values[0]*multiplier


                        stats = pd.Series({'Name': player_name, 'playerID': player_ID, 'Team': team_name, 
                                           'Throws': player_stats['Throws'].tolist()[0], 
                                           'TBF_L': TBF_L,'singles_L': singles_L, 'doubles_L': doubles_L, 'triples_L': triples_L, 
                                           'HR_L': HR_L, 'R_L': R_L, 'ER_L': ER_L, 'BB_L': BB_L, 'HBP_L': HBP_L, 'SO_L': SO_L,  
                                           'TBF_R': TBF_R, 'singles_R': singles_R, 'doubles_R': doubles_R, 'triples_R': triples_R, 
                                           'HR_R': HR_R, 'R_R': R_R, 'ER_R': ER_R, 'BB_R': BB_R, 'HBP_R': HBP_R, 'SO_R': SO_R,
                                           'TBF_Home': TBF_Home,'singles_Home': singles_Home, 'doubles_Home': doubles_Home, 'triples_Home': triples_Home, 
                                           'HR_Home': HR_Home, 'R_Home': R_Home, 'ER_Home': ER_Home, 'BB_Home': BB_Home, 'HBP_Home': HBP_Home, 'SO_Home': SO_Home,  
                                           'TBF_Away': TBF_Away, 'singles_Away': singles_Away, 'doubles_Away': doubles_Away, 'triples_Away': triples_Away, 
                                           'HR_Away': HR_Away, 'R_Away': R_Away, 'ER_Away': ER_Away, 'BB_Away': BB_Away, 
                                           'HBP_Away': HBP_Away, 'SO_Away': SO_Away, position: 1})



                        if(pitcher_df.empty):
                            pitcher_df = pd.DataFrame(columns=pitcher_headers)

                        pitcher_df = pitcher_df.append(stats, ignore_index=True)   # Getting pitcher stats from pitcher stats dataframe
  


        batter_df.fillna(0, inplace=True)
        pitcher_df.fillna(0, inplace=True)

        team_dict.update({team_name: Team_new(team_name, league[team_name], batter_df, pitcher_df, rosters[team_name], 
                                              batter_stats, pitcher_stats, lineup_dict[team_name], lineup_id_dict[team_name])})

    return team_dict, rosters, soup_dict

def new_player_stats(soup, player_name, player_ID, position, arm, team_name):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            soup: fangraphs soup text
            player_ID: ID of player we are getting stats for
            position: position of player
        Returns:
            team_dict: dict with city of team as key and Team instance as value
        """
    
    print("Creating Stats for", player_name)

    for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP', '#ALL']}):
        pos = a_tag.get_text()
        
        if pos==position:

            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'ALL Batters']):
                column_names = ['Name', 'ID', 'PA', 'AVG', 'OBP', 'SLG', 'wOBA', 'Bat', 'BsR', 'Fld', 'WAR']
            else:
                column_names = ['Name', 'ID', 'IP', 'K/9', 'BB/9', 'HR/9', 'BABIP', 'LOB', 'ERA', 'FIP', 'WAR']

            pos_df = pd.DataFrame(columns=column_names)

            table = a_tag.find_next('table')

            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                info = row.a['href']
                elements = info.split('=')
                row_player_ID = elements[1].split('&')[0]
                
                if row_player_ID==player_ID:
                    values_list = []

                    for value in row.findAll('td'):
                        values_list.append(value.get_text())

                        if(len(values_list)==1):
                            values_list.append(player_ID)

                    player_values = pd.Series(values_list, index=column_names)  
                    
                    if position in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']:
                        if(arm=='L'):
                            multiplier = 1.05
                        elif(arm=='R'):
                            multiplier = 1/1.05
                        else:
                            multiplier = 1

                        pa = int(player_values['PA'])
                        obp = float(player_values['OBP'])
                        avg = float(player_values['AVG'])
                        slg = float(player_values['SLG'])

                        on_base = round(obp*pa)
                        hits = on_base
                        PA_Away = pa/(3*multiplier)
                        PA_Home = pa-PA_Away
                        PA_L = pa/(3*multiplier)
                        PA_R = pa-PA_L
                        HBP_Away = PA_Away/100
                        HBP_Home = PA_Home/100
                        HBP_L = PA_L/100
                        HBP_R = PA_R/100
                        SF_Away = PA_Away/180
                        SF_Home = PA_Home/180
                        SF_L = PA_L/180
                        SF_R = PA_R/180

                        for num in range(1, on_base):
                            attempt = on_base-num
                            walks = num
                            average = attempt/(pa-num)
                            if(average<avg):
                                hits = attempt
                                break

                        BB_Away = walks*1/2
                        BB_Home = walks-BB_Away
                        BB_L = walks*5/18
                        BB_R = walks-BB_L
                        AB_Away = PA_Away-BB_Away
                        AB_Home = PA_Home-BB_Home
                        AB_L = PA_L-BB_L
                        AB_R = PA_R-BB_R
                        SO_Away = (pa-on_base)*(.24*multiplier+(.25-avg))
                        SO_Home = (pa-on_base)*(.24/multiplier+(.25-avg))
                        SO_L = (pa-on_base)*(.24*multiplier+(.25-avg))
                        SO_R = (pa-on_base)*(.24/multiplier+(.25-avg))
                        
                        H_Away = hits/2
                        singles_Away = H_Away*.58
                        doubles_Away = H_Away*.22
                        triples_Away = H_Away/70
                        HR_Away = H_Away-singles_Away-doubles_Away-triples_Away
                        temp_slg_Away = (singles_Away+doubles_Away*2+triples_Away*3+HR_Away*4)/AB_Away

                        H_Home = hits-H_Away
                        singles_Home = H_Home*.58
                        doubles_Home = H_Home*.2
                        triples_Home = H_Home/70
                        HR_Home = H_Home-singles_Home-doubles_Home-triples_Home
                        temp_slg_Home = (singles_Home+doubles_Home*2+triples_Home*3+HR_Home*4)/AB_Home
                        
                        H_L = hits/(3*multiplier*multiplier)
                        singles_L = H_L*.58/multiplier
                        doubles_L = H_L*.22/multiplier
                        triples_L = H_L/(70*multiplier)
                        HR_L = H_L-singles_L-doubles_L-triples_L
                        temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L

                        H_R = hits-H_L
                        singles_R = H_R*.58*multiplier
                        doubles_R = H_R*.2*multiplier
                        triples_R = H_R*multiplier/70
                        HR_R = H_R-singles_R-doubles_R-triples_R
                        temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R
                        
                        if(temp_slg_Away>slg/multiplier):
                            while(temp_slg_Away>slg/multiplier and doubles_Away>.2):
                                doubles_Away-=.2
                                singles_Away+=.2
                                temp_slg_Away = (singles_Away+doubles_Away*2+triples_Away*3+HR_Away*4)/AB_Away
                        elif(temp_slg_Away<slg/multiplier):
                            while(temp_slg_Away<slg/multiplier and singles_Away>.2):
                                doubles_Away+=.2
                                singles_Away-=.2
                                temp_slg_Away = (singles_Away+doubles_Away*2+triples_Away*3+HR_Away*4)/AB_Away

                        if(temp_slg_Home>slg*multiplier):
                            while(temp_slg_Home>slg/multiplier and doubles_Home>.2):
                                doubles_Home-=.2
                                singles_Home+=.2
                                temp_slg_Home = (singles_Home+doubles_Home*2+triples_Home*3+HR_Home*4)/AB_Home
                        elif(temp_slg_Home<slg*multiplier):
                            while(temp_slg_Home<slg*multiplier and singles_Home>.2):
                                doubles_Home+=.2
                                singles_Home-=.2
                                temp_slg_Home = (singles_Home+doubles_Home*2+triples_Home*3+HR_Home*4)/AB_Home

                        if(temp_slg_L>slg/multiplier):
                            while(temp_slg_L>slg/multiplier and doubles_L>.2):
                                doubles_L-=.2
                                singles_L+=.2
                                temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L
                        elif(temp_slg_L<slg/multiplier):
                            while(temp_slg_L<slg/multiplier and singles_L>.2):
                                doubles_L+=.2
                                singles_L-=.2
                                temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L

                        if(temp_slg_R>slg*multiplier):
                            while(temp_slg_R>slg/multiplier and doubles_R>.2):
                                doubles_R-=.2
                                singles_R+=.2
                                temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R
                        elif(temp_slg_R<slg*multiplier):
                            while(temp_slg_R<slg*multiplier and singles_R>.2):
                                doubles_R+=.2
                                singles_R-=.2
                                temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R
                                
                        stats_df = pd.DataFrame(columns=['Season', 'Name', 'playerId', 'Tm', 'Bats', 'PA_Away', 'AB_Away',
                                                         'H_Away', 'singles_Away', 'doubles_Away', 'triples_Away', 'HR_Away',
                                                         'BB_Away', 'SO_Away', 'HBP_Away', 'SF_Away', 'PA_Home', 'AB_Home',
                                                         'H_Home', 'singles_Home', 'doubles_Home', 'triples_Home', 'HR_Home',
                                                         'BB_Home', 'SO_Home', 'HBP_Home', 'SF_Home', 'PA_L', 'AB_L', 'H_L',
                                                         'singles_L', 'doubles_L', 'triples_L', 'HR_L', 'BB_L', 'SO_L', 'HBP_L',
                                                         'SF_L', 'PA_R', 'AB_R', 'H_R', 'singles_R', 'doubles_R', 'triples_R',
                                                         'HR_R', 'BB_R', 'SO_R', 'HBP_R', 'SF_R'])

                        stats = pd.Series({'Season': 2021, 'Name': player_name, 'playerId': player_ID, 'Tm': team_name, 'Bats': arm, 
                                           'PA_Away': PA_Away, 'AB_Away': AB_Away, 'H_Away': H_Away, 'singles_Away':singles_Away, 'doubles_Away': doubles_Away, 
                                           'triples_Away': triples_Away, 'HR_Away': HR_Away, 'BB_Away': BB_Away, 'SO_Away': SO_Away, 'HBP_Away': HBP_Away, 'SF_Away': SF_Away,
                                           'PA_Home': PA_Home, 'AB_Home': AB_Home, 'H_Home': H_Home, 'singles_Home': singles_Home, 'doubles_Home': doubles_Home, 
                                           'triples_Home': triples_Home, 'HR_Home': HR_Home, 'BB_Home': BB_Home, 'SO_Home': SO_Home, 'HBP_Home': HBP_Home, 'SF_Home': SF_Home,
                                           'PA_L': PA_L, 'AB_L': AB_L, 'H_L': H_L, 'singles_L':singles_L, 'doubles_L': doubles_L, 
                                           'triples_L': triples_L, 'HR_L': HR_L, 'BB_L': BB_L, 'SO_L': SO_L, 'HBP_L': HBP_L, 'SF_L': SF_L, 
                                           'PA_R': PA_R, 'AB_R': AB_R, 'H_R': H_R, 'singles_R': singles_R, 'doubles_R': doubles_R, 
                                           'triples_R': triples_R,'HR_R': HR_R, 'BB_R': BB_R, 'SO_R': SO_R, 'HBP_R': HBP_R,'SF_R': SF_R}) 
                        
                        stats_df = stats_df.append(stats, ignore_index=True)
                        
                        return stats_df

                    elif(pos in ['SP', 'RP']):   
                        
                        if(arm=='L'):
                            multiplier = 1.05
                        elif(arm=='R'):
                            multiplier = 1/1.05
                        else:
                            multiplier = 1

                        ip = int(round(float(player_values['IP'])))
                        ip_Away = ip/2
                        ip_Home = ip-ip_Away
                        ip_L = ip*.4
                        ip_R = ip-ip_L
                        outs_Away = ip_Away*3
                        outs_Home = ip_Home*3
                        outs_L = ip_L*3
                        outs_R = ip_R*3
                        SO_Away = float(player_values['K/9'])*ip_Away/9
                        SO_Home = float(player_values['K/9'])*ip_Home/9
                        SO_L = float(player_values['K/9'])*ip_L*multiplier/9
                        SO_R = float(player_values['K/9'])*ip_L/(multiplier*9)
                        ER_Away = ip_Away*float(player_values['ERA'])/9
                        ER_Home = ip_Home*float(player_values['ERA'])/9
                        ER_L = ip_L*float(player_values['ERA'])/(multiplier*9)
                        ER_R = ip_R*float(player_values['ERA'])*multiplier/9
                        R_Away = ER_Away*1.07
                        R_Home = ER_Home*1.07
                        R_L = ER_L*1.07
                        R_R = ER_R*1.07
                        ERA_Away = ER_Away*27/outs_Away
                        ERA_Home = ER_Home*27/outs_Home
                        ERA_L = ER_L*27/outs_L
                        ERA_R = ER_R*27/outs_R
                        BB_Away = float(player_values['BB/9'])*ip_Away/9
                        BB_Home = float(player_values['BB/9'])*ip_Home/9
                        BB_L = float(player_values['BB/9'])*ip_L/(multiplier*9)
                        BB_R = float(player_values['BB/9'])*ip_R*multiplier/9
                        temp_whip_Away = ERA_Away/10 + .8
                        temp_whip_Home = ERA_Home/10 + .8
                        temp_whip_L = ERA_L/(10*multiplier) + .8
                        temp_whip_R = ERA_R*multiplier/10 + .8
                        temp_wh_Away = temp_whip_Away*ip_Away
                        temp_wh_Home = temp_whip_Home*ip_Home
                        temp_wh_L = temp_whip_L*ip_L
                        temp_wh_R = temp_whip_R*ip_R
                        HR_Away =  float(player_values['HR/9'])*ip_Away/9
                        HR_Home = float(player_values['HR/9'])*ip_Home/9
                        HR_L =  float(player_values['HR/9'])*ip_L/(multiplier*9)
                        HR_R = float(player_values['HR/9'])*ip_R*multiplier/9
                        triples_Away = outs_Away*.005
                        triples_Home = outs_Home*.005
                        triples_L = outs_L*.005
                        triples_R = outs_R*.005
                        doubles_Away = outs_Away/17
                        doubles_Home = outs_Home/17
                        doubles_L = outs_L/(17*multiplier)
                        doubles_R = outs_R*multiplier/17   
                        singles_Away = temp_wh_Away-HR_Away-triples_Away-doubles_Away-BB_Away
                        singles_Home = temp_wh_Home-HR_Home-triples_Home-doubles_Home-BB_Home
                        singles_L = temp_wh_L-HR_L-triples_L-doubles_L-BB_L
                        singles_R = temp_wh_R-HR_R-triples_R-doubles_R-BB_R
                        TBF_Away = .98*(outs_Away+temp_wh_Away)
                        TBF_Home = .98*(outs_Home+temp_wh_Home)
                        TBF_L = .98*(outs_L+temp_wh_L)
                        TBF_R = .98*(outs_R+temp_wh_R)
                        H_Away = singles_Away+doubles_Away+triples_Away+HR_Away
                        H_Home = singles_Home+doubles_Home+triples_Home+HR_Home
                        H_L = singles_L+doubles_L+triples_L+HR_L
                        H_R = singles_R+doubles_R+triples_R+HR_R
                        HBP_Away = TBF_Away*.01
                        HBP_Home = TBF_Home*.01
                        HBP_L = TBF_L*.01
                        HBP_R = TBF_R*.01
                        
                        stats_df = pd.DataFrame(columns=['Season', 'Name', 'playerId', 'Tm', 'Throws', 'TBF_Away', 'ERA_Away',
                                                         'H_Away', 'singles_Away', 'doubles_Away', 'triples_Away', 'HR_Away', 'R_Away',
                                                         'ER_Away', 'BB_Away', 'SO_Away', 'HBP_Away', 'TBF_Home', 'ERA_Home',
                                                         'H_Home', 'singles_Home', 'doubles_Home', 'triples_Home', 'HR_Home', 'R_Home',
                                                         'ER_Home', 'BB_Home', 'SO_Home', 'HBP_Home', 'PA_L', 'AB_L', 'H_L',
                                                         'TBF_L', 'ERA_L','H_L', 'singles_L', 'doubles_L', 'triples_L', 'HR_L', 'R_L',
                                                         'ER_L', 'BB_L', 'SO_L', 'HBP_L', 'TBF_R', 'ERA_R',
                                                         'H_R', 'singles_R', 'doubles_R', 'triples_R', 'HR_R', 'R_R',
                                                         'ER_R', 'BB_R', 'SO_R', 'HBP_R'])


                        stats = pd.Series({'Season': 2021, 'Name': player_name, 'playerId': player_ID, 'Tm': team_name, 'Throws': arm, 
                                           'TBF_Away': TBF_Away, 'ERA_Away': ERA_Away, 'H_Away': H_Away, 'singles_Away': singles_Away, 'doubles_Away': doubles_Away, 
                                           'triples_Away': triples_Away, 'HR_Away': HR_Away, 'R_Away': R_Away, 'ER_Away': ER_Away, 'BB_Away': BB_Away, 'SO_Away': SO_Away, 'HBP_Away': HBP_Away,
                                           'TBF_Home': TBF_Home, 'ERA_Home': ERA_Home, 'H_Home': H_Home, 'singles_Home': singles_Home, 'doubles_Home': doubles_Home, 
                                           'triples_Home': triples_Home, 'HR_Home': HR_Home, 'R_Home': R_Home, 'ER_Home': ER_Home, 'BB_Home': BB_Home, 'SO_Home': SO_Home, 'HBP_Home': HBP_Home,
                                           'TBF_L': TBF_L, 'ERA_L': ERA_L, 'H_L': H_L, 'singles_L': singles_L, 'doubles_L': doubles_L, 
                                           'triples_L': triples_L, 'HR_L': HR_L, 'R_L': R_L, 'ER_L': ER_L, 'BB_L': BB_L, 'SO_L': SO_L, 'HBP_L': HBP_L,
                                           'TBF_R': TBF_R, 'ERA_R': ERA_R, 'H_R': H_R, 'singles_R': singles_R, 'doubles_R': doubles_R, 
                                           'triples_R': triples_R, 'HR_R': HR_R, 'R_R': R_R, 'ER_R': ER_R, 'BB_R': BB_R, 'SO_R': SO_R, 'HBP_R': HBP_R})
                        
                        stats_df = stats_df.append(stats, ignore_index=True)
                        
                        return stats_df               
                        
    return None                    
                                           
def active_roster(name):
    
    headers = {'User-Agent': header_name}

    url_list = []
    url_start = 'https://www.mlb.com/'
    
    team_roster_dict = {}
    hand_dict = {}
    
    if name[-1]=='2':
        name = name[:len(name)-1]
    
   # for team_name in teams:
    team_name = city_to_nickname_dict[name]   
    mlb_name = None

    # Making sure all team names are in mlb.com format
    if team_name=='red sox':
        mlb_name = 'redsox'
    elif team_name=='blue jays':
        mlb_name = 'bluejays'
    elif team_name=='d-backs':
        mlb_name = 'dbacks'
    elif team_name=='white sox':
        mlb_name = 'whitesox'
    else:
        mlb_name = team_name

    url = url_start+mlb_name+'/roster/depth-chart'
    position_player_dict = {}

    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    # Getting player names on each depth chart
    for table in soup.findAll('table', attrs={'class': 'roster__table'}):
        position = table.td.get_text()

        if position=='Rotation':
            position = 'SP'
        elif position=='Bullpen':
            position = 'RP'
        elif position=='Catcher':
            position = 'C'
        elif position=='First Base':
            position = '1B'
        elif position=='Second Base':
            position = '2B'
        elif position=='Shortstop':
            position = 'SS'
        elif position=='Third Base':
            position = '3B'
        elif position=='Left Field':
            position = 'LF'
        elif position=='Center Field':
            position = 'CF'
        elif position=='Right Field':
            position = 'RF'
        elif position=='Designated Hitter':
            position = 'DH'

        #position_player_dict[position] = []

        for row in table.tbody.findAll('td', attrs={'class': 'info'}):
            if row.find('span', attrs={'class': 'status-il'}):
                continue 

            player_name = unidecode.unidecode(row.a.get_text().strip())
            if position in ['SP', 'RP']:
                hand_dict[player_name] = row.findNext('td').get_text().strip().split('/')[1]
            else:
                hand_dict[player_name] = row.findNext('td').get_text().strip().split('/')[0]
            #position_player_dict[position].append(player_name)

            if row.find('span', attrs={'class': 'closer'}):
                position_player_dict['Closer'] = player_name 
                continue


    time.sleep(1)

    url = url_start+mlb_name+'/roster'

    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    # Getting player names on each roster
    for table in soup.findAll('table', attrs={'class': 'roster__table'}):
        position = table.td.get_text()

        if position in ('Catchers', 'Infielders', 'Outfielders', 'Designated Hitter' ,'Designated Hitters'):
            position = 'Batters'

        if position not in position_player_dict.keys():
            position_player_dict[position] = []

        for row in table.tbody.findAll('td', attrs={'class': 'info'}):

            player_name = unidecode.unidecode(row.a.get_text().strip())
            position_player_dict[position].append(player_name)
            
    if name=='Los Angeles Angels':   # Adding Shohei Ohtani to Batter list
        position_player_dict['Batters'].append('Shohei Ohtani')


    team_roster_dict[nickname_to_city_dict[team_name]] = position_player_dict

    time.sleep(1)
                        
    return position_player_dict, hand_dict

def closers():
    
    headers = {'User-Agent': header_name}

    url_list = []
    url_start = 'https://www.mlb.com/'
    
    team_closer_dict = {}
    
    for team_name in mlb_teams:
        
        mlb_name = None

        # Making sure all team names are in mlb.com format
        if team_name=='red sox':
            mlb_name = 'redsox'
        elif team_name=='blue jays':
            mlb_name = 'bluejays'
        elif team_name=='d-backs':
            mlb_name = 'dbacks'
        elif team_name=='white sox':
            mlb_name = 'whitesox'
        else:
            mlb_name = team_name
     
        url = url_start+mlb_name+'/roster/depth-chart'
        position_player_dict = {}

        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        # Getting player names on each depth chart
        for table in soup.findAll('table', attrs={'class': 'roster__table'}):
            position = table.td.get_text()

            if position=='Bullpen':
                position = 'RP'
            else:
                continue

            for row in table.tbody.findAll('td', attrs={'class': 'info'}):
                if row.find('span', attrs={'class': 'status-il'}):
                    continue 
                
                player_name = unidecode.unidecode(row.a.get_text().strip())
                
                if row.find('span', attrs={'class': 'closer'}):
                    position_player_dict['Closer'] = player_name 
                        
        team_closer_dict[nickname_to_city_dict[team_name]] = position_player_dict

        #time.sleep(1)
        
                     
    return team_closer_dict

def game_predictions(schedule_dict, team_dict, teams_list=[], new_lineups=[], reset=True):
    games = 1000
    team_wins_dict = {}
    team_stats_dict = {}
    
    for matchups in schedule_dict.values():
        for matchup in matchups:
            if matchup[0] not in teams_list or matchup[1] not in teams_list:
                continue
            
            if(len(lineup_dict[matchup[0]]['lineup'])<1 or len(lineup_dict[matchup[0]]['lineup'])<1):   # Only predict games with lineups for both teams
                continue
            for n in range(games):
                innings=9

                if(matchup[0][-1]=='2' or matchup[0]+'2' in team_dict.keys()):   # 7 inning double headers
                    innings = 7
                
                g = Game_new(team_dict[matchup[0]], team_dict[matchup[1]], innings)
                g.play()
                g.print_result()
                team_dict[matchup[0]].restart(season=False)
                team_dict[matchup[1]].restart(season=False)
            #print(matchup[0] + " win rate", team_dict[matchup[0]].wins/games)
            #print(matchup[0] +  " R/G", team_dict[matchup[0]].cum_runs/games)
            #print(matchup[1] + " wins rate", team_dict[matchup[1]].wins/games)
            #print(matchup[1] +  " R/G", team_dict[matchup[1]].cum_runs/games)
            Away_o, Away_p = team_dict[matchup[0]].cumul_stats_df()
            Home_o, Home_p = team_dict[matchup[1]].cumul_stats_df()
            
            
            team_stats_dict[matchup[0]] = {}
            team_stats_dict[matchup[1]] = {}
            
            team_stats_dict[matchup[0]]['Batting'] = Away_o
            team_stats_dict[matchup[0]]['Pitching'] = Away_p
            team_stats_dict[matchup[1]]['Batting'] = Home_o
            team_stats_dict[matchup[1]]['Pitching'] = Home_p
            #print(Away_o)
            #print(Away_p)
            #print(Home_o)
            #print(Home_p)
            #print(" ")

            
            team_wins_dict[matchup[0]] = round(team_dict[matchup[0]].wins/team_dict[matchup[0]].game_no, 2)
            team_wins_dict[matchup[1]] = round(team_dict[matchup[1]].wins/team_dict[matchup[0]].game_no, 2)

            if reset:   # Resetting player stats
                team_dict[matchup[0]].restart(season=True)
                team_dict[matchup[1]].restart(season=True)

    return team_wins_dict, team_stats_dict

def moneylines_and_predictions(schedule_dict, team_dict, team_stats_dict, new_lineups, year):
    """Loads moneylines to spreadsheet
    Args:
        scedule_dict: dict of games to be played (date as key, matchups as values)
        year: year that will be used to gather the dates of the games
    Returns:
        dataframe of data with moneylines
    """
    names = ['Open', 'ML','Westgate','MGM Mirage', 'betMGM',
             'William Hill', 'CG Technology', 'Circa Sports','Stations']
    book = pd.DataFrame(columns=names)
    temp_away_list = []
    temp_home_list = []
    teams_list = []
    dates_list = []
    times_list = []
    
    file_name = 'Baseball Picks.csv'
    current_spreadsheet = getting_spreadsheet(file_name) # Retrieving Spreadsheet
    current_spreadsheet['Date'] = current_spreadsheet['Date'].apply(lambda date: spreadsheet_date_formatter(date))
    current_spreadsheet['Date'] = current_spreadsheet['Date'].replace('', np.nan).ffill()
    current_spreadsheet['Time'] = current_spreadsheet['Time'].replace('', np.nan).ffill()
    
    url = 'http://www.vegasinsider.com/mlb/odds/las-vegas/'
    headers = {'User-Agent': header_name}

    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
        for num, games in enumerate(gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'})):
            for matchup in games.findAll('tr'):
                for game_info in matchup.findAll('td', attrs={'class': ['viCellBg1 cellTextNorm cellBorderL1 gameCell',
                                                                      'viCellBg2 cellTextNorm cellBorderL1 gameCell']}):

                    no_game = False

                    for date in game_info.findAll('span', attrs={'class': 'cellTextHot'}):   # Finding and formatting dates
                        game_date = date.get_text().split()[0]
                        month = game_date.split('/')[0]
                        day = game_date.split('/')[1]
                        game_date = str(month) + '/' + str(day) + '/' + str(year)
                        dates_list.extend([game_date, game_date])

                        time = date.get_text().split()[1].strip()

                        if date.get_text().split()[2].strip()=='PM' and int(time.split(':')[0])<12:
                            game_time = str(int(time.split(':')[0])+12)
                        else:
                            game_time = str(int(time.split(':')[0]))

                        times_list.extend([game_time, game_time])

                    for team_name in game_info.findAll('a', attrs={'class': 'tabletext'}): # Gathering team names
                        if(team_name.get_text().strip()=='N.Y. Yankees'):
                            teams_list.append('New York Yankees')
                        elif(team_name.get_text().strip()=='N.Y. Mets'):
                            teams_list.append('New York Mets')
                        elif(team_name.get_text().strip()=='Chi. Cubs'):
                            teams_list.append('Chicago Cubs')
                        elif(team_name.get_text().strip()=='Chi. White Sox'):
                            teams_list.append('Chicago White Sox')
                        elif(team_name.get_text().strip()=='L.A. Dodgers'):
                            teams_list.append('Los Angeles Dodgers')
                        elif(team_name.get_text().strip()=='L.A. Angels'):
                            teams_list.append('Los Angeles Angels')
                        else:
                            teams_list.append(team_name.get_text().strip())                  
                            
                    if(teams_list[-2] in teams_list[0:len(teams_list)-2]):   # Double headers
                        temp1 = teams_list[-2]
                        temp2 = teams_list[-2]
                        teams_list = teams_list[0:len(teams_list)-2]
                        teams_list.append(temp1+'2')
                        teams_list.append(temp2+'2')

                    matchups = list(chain.from_iterable(schedule_dict.values()))  

                    if((teams_list[-2], teams_list[-1]) not in matchups
                       or (len(lineup_dict[teams_list[-2]]['lineup'])<1 or len(lineup_dict[teams_list[-1]]['lineup'])<1)
                       or len(current_spreadsheet.loc[(current_spreadsheet['Team']==teams_list[-2]) & (current_spreadsheet['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))])>0
                       or len(current_spreadsheet.loc[(current_spreadsheet['Team']==teams_list[-1]) & (current_spreadsheet['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))])>0
                       or teams_list[-2] not in team_dict.keys() or teams_list[-1] not in team_dict.keys()):   # Games not being played or without lineups
         
                        teams_list = teams_list[0:len(teams_list)-2]
                        dates_list = dates_list[0:len(dates_list)-2]
                        times_list = times_list[0:len(times_list)-2]
                        no_game = True

                    if(not no_game):
                        for num, spread in enumerate(matchup.findAll('td', attrs={'class': 
                                                                     ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap oddsCell',
                                                                      'viCellBg1 cellTextHot cellBorderL1 center_text nowrap oddsCell',
                                                                      'viCellBg2 cellTextNorm cellBorderL1 center_text nowrap oddsCell',
                                                                      'viCellBg2 cellTextHot cellBorderL1 center_text nowrap oddsCell']})):           
                            spread_text = spread.get_text().strip()

                            spread_away = str(spread).split('<br/>')[1]
                            if(len(spread_away.split())==0 or spread_away.split()[0]=='XX'):
                                away_spread = ''
                            else:   
                                away_spread = spread_away.split()[0]

                            spread_home = str(spread).split('<br/>')[2]
                            if(len(spread_home.split())<2 or spread_home.split()[0]=='XX'):
                                home_spread = ''
                            else:   
                                home_spread = spread_home.split()[0]


                            temp_away_list.append(away_spread)
                            temp_home_list.append(home_spread)

                            if len(temp_home_list) == 9:
                                book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                                book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                                temp_away_list = []
                                temp_home_list = []

    while(len(dates_list) > len(teams_list)):  # In case extra games in odds like All Star game
        dates_list.pop()
        times_list.pop()
        book.drop(book.tail(1).index, inplace=True)

    
    book['Team'] = pd.Series(teams_list)
    
    book['Date'] = dates_list
    book['Time'] = times_list
    book['Version'] = '1.2'
    book = book[book.Date==datetime.datetime.now().strftime('%m/%d/%Y')]    # Only getting odds for games in current week
    
    team_wins_dict, team_stats_dict = game_predictions(schedule_dict, team_dict, teams_list, new_lineups)

    book['Implied Probability'] = book['ML'].apply(lambda x: implied_probability(x))
    book['Jack Prediction'] = book['Team'].apply(lambda x: team_wins_dict[x])
    book['Jack Pick'] = book.apply(lambda x: ml_pick(x), axis=1)
    book['Winner'] = np.nan
    book['Result'] = np.nan
    book['Outcome'] = np.nan
    book['Total'] = np.nan
    
    book['Team'] = book['Team'].apply(lambda team_name: team_name if team_name[-1]!='2' else team_name[:len(team_name)-1])

    odds = book[['Version', 'Date', 'Time', 'Team', 'ML', 'Implied Probability', 'Jack Prediction', 'Jack Pick', 'Winner', 'Result', 'Outcome', 'Total']]
    #odds.set_index('Team', inplace=True)
    
    return odds, team_stats_dict

In [ ]:
def getting_spreadsheet(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))

def separating_games(df):
    """Separates old games and new games
    Args:
        df: dataframe of data
    Returns:
        old_games: games with results already
        current_game: games without results
    """
    old_games = df[(df['Result'] != '') | (df['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))]
    old_games_index =  df[(df['Result'] != '') | (df['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))].index 
    current_games = df.drop(old_games_index)
    current_games['Team'].replace('', np.nan, inplace=True)
    current_games.dropna(subset=['Team'], inplace=True)
    
    return old_games, current_games

def spreadsheet_date_formatter(date):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    if date in (np.nan, ''):
        return(date)
    else:
        split_date = date.split('/')
        day = split_date[1]
        day = day.zfill(2)
        date = split_date[0] +  '/' + day + '/' + split_date[2]
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        date = date.strftime('%m/%d/%Y')

        return(date)
    
def date_ffill_formatter(df):
    """Formats dates to be in every other row
    Args:
        df: df of data
    """
    for index, row in df.iterrows():
        if index%2 == 0:
            continue
        else:
            df.at[index, 'Date'] = ''   
            df.at[index, 'Time'] = ''
            df.at[index, 'Version'] = ''  
            
def combining_data(df_top, df_bottom):
    """Appends two dataframes
    Args:
        df_top: dataframe of data to go on top
        df_bottom: dataframe of data to go on bottom
    Returns:
        df: combined dataframe
    """
    cols = df_top.columns
    df = pd.concat([df_top, df_bottom], ignore_index=True, )
    df = df[cols]
    df.replace(np.nan, '', inplace=True)
    return(df)  

def game_outcomes(current_games, game_dates, teams):
    """Finds winners and losers of games
    Args:
        df: dataframe of data
        game_dates: dates of games completed
        teams: teams in games that are completed
    Returns:
        winners: dict with dates as keys and a list of winners as values
        losers: dict with dates as keys and a list of losers as values
    """
    year = 2021
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2021-mlb-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    skip_game = False
    
    current_games.loc[current_games['Date']!=datetime.datetime.now().strftime('%m/%d/%Y'), 'Result'] = 'ppd'  # Start with ppd then chagne if game had outcome
    game_time = -1
    
    for games in soup.findAll('tbody'):
        for game in games.findAll('tr', attrs={'class': 'tr'}):
            
            if skip_game:
                skip_game = False
                continue
            
            if game.find('span', attrs={'class': 'time'}):
                time = game.find('span', attrs={'class': 'time'}).get_text().strip().split('.m.')[0]

                if time[-1]=='p':
                    if ':' in time:
                        game_time = str(int(time.split(':')[0])+12)
                    else: 
                        game_time = str(int(time.split()[0])+12)
                else:
                    if ':' in time:
                        game_time = str(int(time.split(':')[0]))
                    else: 
                        game_time = str(int(time.split()[0]))

            if game.find('span', attrs={'class': 'day short'}):
                date = game.find('span', attrs={'class': 'day short'}).get_text()
                m = date.split('/')[0]
                d = date.split('/')[1]
                date = m + ' ' + d + ' ' + str(year)
                date = datetime.datetime.strptime(date, '%m %d %Y')
                year = date.year
                month = date.month
                day = date.day
                date = date.strftime('%m/%d/%Y')

                if date not in game_dates or date==datetime.datetime.now().strftime('%m/%d/%Y') :   # Skip games that don't need to be updated with scores
                    skip_game = True
                    continue
                
                away_team = nickname_to_city_dict[game.find('span', attrs={'class': 'team-name long'}).get_text().strip().lower()]
                
                away_row_index = current_games.index[(current_games['Date']==date) & (current_games['Time']==int(game_time)) & (current_games['Team']==away_team)]
                #current_games.at[away_row_index, 'Time'] = game_time

                if away_team not in teams:  # Skip games that don't need to be updated with scores
                    skip_game = True
                    continue

                away_score = game.find('td', attrs={'class': ['td number td-number score',
                                                              'td number td-number score is-winner']}).get_text().strip()

                current_games.at[away_row_index, 'Score'] = away_score

                if game.find('td', attrs={'class': 'td number td-number score is-winner'}):
                    current_games.at[away_row_index, 'Result'] = 'w'
                    current_games.at[away_row_index, 'Winner'] = away_team
                else:
                    current_games.at[away_row_index, 'Result'] = 'l'
                    current_games.at[away_row_index, 'Winner'] = ''
            else:
                home_team = nickname_to_city_dict[game.find('span', attrs={'class': 'team-name long'}).get_text().strip().lower()]
                home_row_index = current_games.index[(current_games['Date']==date) & (current_games['Time']==int(game_time)) & (current_games['Team']==home_team)]
                
                home_score = game.find('td', attrs={'class': ['td number td-number score',
                                                              'td number td-number score is-winner']}).get_text().strip()
                current_games.at[home_row_index, 'Score'] = home_score
                
                if game.find('td', attrs={'class': 'td number td-number score is-winner'}):
                    current_games.at[home_row_index, 'Result'] = 'w'
                    current_games.at[home_row_index, 'Winner'] = home_team
                else:
                    current_games.at[home_row_index, 'Result'] = 'l'
                    current_games.at[home_row_index, 'Winner'] = ''
                 
    return current_games 

def implied_probability(ML):
    """Uses odds to determine implied probability
    Args:
        row: row of data from dataframe
    Returns:
        implied win probability if it exists
    """

    if ML[0] == '+':
        return(round(100.0/(100+int(ML[1:])), 2))
    elif ML[0] == '-':
        return(round(int(ML[1:])/(100.0+int(ML[1:])), 2))
    else:
        return('')      
    
def ml_pick(row):
    """Uses win% and odds to determine what team to pick
    Args:
        row: row of data from dataframe
        predictor: name of predicting column
    Returns:
        pick if there is one
    """
    
    if row['Implied Probability'] == '':
        return('')
    elif (row['Jack Prediction'] > row['Implied Probability']):
        return(row.Team)
    else:
        return('')      
    
def spreadsheet_formatter(df):
    """Formats spreadsheet
    Args:
        df: dataframe of data
    """
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)
        
    df.replace(np.nan, '', inplace=True)  
    
def ml_formatter(row):
    """Formats ml
    Args:
        row: row of data from dataframe
    Returns:
        formatted odds
    """
    
    if row['ML'] == '':
        return(row['ML'])
    elif str(row['ML'])[0] == '+':
        return(str(row['ML']))
    elif int(row['ML']) >= 100:
        return('+' + str(int(row['ML'])))
    if int(row['ML']) <= -100:
        return('-' + str(int(abs(row['ML']))))
    else:
        return(str(row['ML']))
        
def team_won_lost(row, winners, losers):
    """Determines team outcomes
    Args:
        row: row of data from dataframe
        winners: dict with dates as keys and a list of winners as values
        losers: dict with dates as keys and a list of losers as values
    Returns:
        updated entry for winner/loser
    """
    if row.date != '':
        if row.date in winners.keys():
            if row.team in winners[row.date]:
                return('w')
            elif row.team in losers[row.date]:
                return('l')
            else:
                return('')
        else:
            return('')
    else:
        return('')   
    
def money_won_lost(row):
    """Determines amount won lost
    Args:
        row: row of data from dataframe
    Returns:
        amount won/lost
    """
    if(row['Jack Pick'] not in [None, '']):
        if row['Result'] == 'w':
            if row['ML'][0] == '+':
                return(int(row['ML'][1:]))
            else:
                return(100)
        elif row['Result'] == 'l':
            if row['ML'][0] == '+':
                return(-100)
            else:
                return(int(row['ML']))
        else:
            return(int(0))   
    else:
        return(int(0))

def writing_spreadsheet(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)

In [ ]:
batter_stats = reading_data('Joined_Batter_Stats.csv')
batter_stats = batter_stats.replace(np.nan, 0)
pitcher_stats = reading_data('Joined_Pitcher_Stats.csv')
pitcher_stats = pitcher_stats.replace(np.nan, 0)
lineup_dict = {}
lineup_id_dict = {}
team_dict = {}
team_stats_dict = {}
mlb_rosters = {}
rosters = {}
team_hand_dict = {}
soup_dict = {}

In [ ]:
lineup_dict, lineup_id_dict, mlb_rosters, team_hand_dict, schedule_dict, new_lineups  = fangraphs_daily_lineups(lineup_dict)
team_dict, rosters, soup_dict = create_teams_new(batter_stats, pitcher_stats, lineup_dict, lineup_id_dict, team_dict, mlb_rosters, rosters, team_hand_dict, new_lineups, soup_dict)
#new_games, team_stats_dict = moneylines_and_predictions(schedule_dict, team_dict, team_stats_dict, year)
#game_predictions(schedule_dict_temp, team_dict, team_dict.keys(), team_dict.keys(), False)

In [ ]:
file_name = 'Baseball Picks.csv'
date = datetime.date.today().strftime('%m/%d/%Y')
year = 2021

df = getting_spreadsheet(file_name) # Retrieving Spreadsheet
df['Date'] = df['Date'].apply(lambda date: spreadsheet_date_formatter(date))
df['Date'] = df['Date'].replace('', np.nan).ffill()
df['Time'] = df['Time'].replace('', np.nan).ffill()
spreadsheet_formatter(df)
df['ML'] = df.apply(lambda row: ml_formatter(row), axis=1)
old_games, current_games = separating_games(df)


if not current_games.empty:
    current_games = game_outcomes(current_games, current_games.Date.unique(), current_games.Team.unique())
    current_games['Outcome'] = current_games.apply(lambda row: money_won_lost(row), axis=1)
    df = combining_data(old_games, current_games)
  
df['Date'] = df['Date'].apply(lambda date: spreadsheet_date_formatter(date))

#if(date > df.Date.max() or len(df.loc[df['Date']==df.Date.max()])<30*len(list(schedule_dict.values())[0])):
#lineup_dict, lineup_id_dict, mlb_rosters, team_hand_dict, schedule_dict, new_lineups  = fangraphs_daily_lineups(lineup_dict)
#team_dict, rosters, soup_dict = create_teams_new(batter_stats, pitcher_stats, lineup_dict, lineup_id_dict, team_dict, mlb_rosters, rosters, new_lineups, soup_dict)
new_games, team_stats_dict = moneylines_and_predictions(schedule_dict, team_dict, team_stats_dict, new_lineups, year)
df = combining_data(df, new_games)

df['Total'] = int(df.loc[(df['Outcome'] != '') & (df['Outcome'] != np.nan)]['Outcome'].sum())
date_ffill_formatter(df)
#print(df)

#writing_spreadsheet(df, file_name)

In [ ]:
df.tail(30)

In [ ]:
writing_spreadsheet(df, file_name)

In [ ]:
team_stats_dict['Oakland']['Batting']

In [ ]:
game_predictions(schedule_dict, team_dict, team_dict.keys(), team_dict.keys(), False)

In [ ]:
team_dict['Atlanta'].restart(season=True)
team_dict['Philadelphia'].restart(season=True)

In [ ]:
for num in range(1000):
    g = Game_new(team_dict['Oakland'], team_dict['San Diego'], 9)
    g.play()
    g.print_result()
    team_dict['Oakland'].restart(season=False)
    team_dict['San Diego'].restart(season=False)

In [ ]:
team_dict['Toronto'].wins

In [ ]:
a, b = team_dict['Oakland'].cumul_stats_df()
a

In [ ]:
mlb_rosters

In [ ]:
file_name = 'Baseball Picks.csv'
current_spreadsheet = getting_spreadsheet(file_name) # Retrieving Spreadsheet
current_spreadsheet['Date'] = current_spreadsheet['Date'].apply(lambda date: spreadsheet_date_formatter(date))
current_spreadsheet['Date'] = current_spreadsheet['Date'].replace('', np.nan).ffill()
current_spreadsheet['Time'] = current_spreadsheet['Time'].replace('', np.nan).ffill()

len(current_spreadsheet.loc[(current_spreadsheet['Team']=='Los Angeles Dodgers') 
                                                                   & (current_spreadsheet['Date']==datetime.datetime.now().strftime('%m/%d/%Y'))
                                                                   & (current_spreadsheet['Time']<int(23))])